In [2]:
import pandas as pd
import numpy as np
import matplotlib 
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler

# 통계 분석 패키지
from scipy import stats
import scipy.stats

from scipy.stats import shapiro # 정규성 검증
from statsmodels.formula.api import ols # 회귀분석
# proportion test 패키지
from statsmodels.stats.proportion import proportions_ztest
from scipy.stats import chi2_contingency #카이제곱 검정
from scipy.stats import bartlett

# 주성분 분석
from sklearn.decomposition import PCA
# 회귀분석
import statsmodels.api as sm

# linkage: 계층형 군집분석, dendrogram: 계층구조 나무, cluster: linkage로부터 얻은 값으로 군집 생성
from scipy.cluster.hierarchy import linkage, dendrogram, fcluster
# 군집 방법 : AgglomerativeClustering(계층형), KMeans(k-평균), DBSCAN
from sklearn.cluster import AgglomerativeClustering, KMeans, DBSCAN

# 실루엣(Silhouette) 계수 산출
from sklearn.metrics import silhouette_samples, silhouette_score
# 실루엣 계수 산출 및 시각화 라이브러리 (pip install yellowbrick 실행)
from yellowbrick.cluster import SilhouetteVisualizer

# 데이터 전처리 : 항목 값에 대한 index 생성
from mlxtend.preprocessing import TransactionEncoder
# 지지도 계산
from mlxtend.frequent_patterns import apriori
# 연관 규칙
from mlxtend.frequent_patterns import association_rules

In [3]:
%matplotlib inline

#import matplotlib.pyplot as plt

# 맑은 고딕 적용
matplotlib.rc("font", family = "NaNumGothic")

# 음수 표시
# -값 표시
matplotlib.rc('axes', unicode_minus=False)

In [44]:
df_orders=pd.read_csv("./on_orders.csv", encoding="cp949")
df_orders

,idUser,idOrder,OrderDT,ItemCode,Price,DeliveryDT
0,U10001,U10001-O2021-1002,07JAN21:17:08:51,L4-M17-S0530-1024,33310,08JAN2021:06:24:00
1,U10001,U10001-O2021-1002,07JAN21:17:08:51,L1-M21-S0540-1082,3780,08JAN2021:06:24:00
2,U10001,U10001-O2021-1002,07JAN21:17:08:51,L1-M15-S0140-1311,22520,08JAN2021:06:24:00
3,U10001,U10001-O2021-1002,07JAN21:17:08:51,L4-M12-S0350-1035,21630,08JAN2021:06:24:00
4,U10001,U10001-O2021-1003,14JAN21:16:50:14,L4-M12-S0640-1057,11700,15JAN2021:06:28:00
...,...,...,...,...,...,...
856618,U13000,U13000-O2023-1008,28DEC23:14:06:13,L4-M12-S0350-1063,11520,29DEC2023:04:23:00
856619,U13000,U13000-O2023-1008,28DEC23:14:06:13,L1-M12-S0330-1011,18870,29DEC2023:04:23:00
856620,U13000,U13000-O2023-1008,28DEC23:14:06:13,L1-M21-S0540-1161,3120,29DEC2023:04:23:00
856621,U13000,U13000-O2023-1008,28DEC23:14:06:13,L1-M17-S0030-1016,37760,29DEC2023:04:23:00


In [45]:
df_items=pd.read_csv("./on_items.csv", encoding="cp949")
df_items

,ItemLargeCode,ItemLargeName,ItemMiddleCode,ItemMiddleName,ItemSmallCode,ItemSmallName,ItemCode,ItemName,PriceYear,PriceMin,PriceMax
0,L1,가공식품,M11,곡물,S0080,국수,L1-M11-S0080-1001,(식품)샘표 김치국수 101g 10입,2021,15840,16130
1,L1,가공식품,M11,곡물,S0080,국수,L1-M11-S0080-1001,(식품)샘표 김치국수 101g 10입,2022,17030,17340
2,L1,가공식품,M11,곡물,S0080,국수,L1-M11-S0080-1001,(식품)샘표 김치국수 101g 10입,2023,17380,18640
3,L1,가공식품,M11,곡물,S0080,국수,L1-M11-S0080-1002,2.1kg 6배 메밀 Bestco 희석용 소바 국수장국,2022,14160,15350
4,L1,가공식품,M11,곡물,S0080,국수,L1-M11-S0080-1002,2.1kg 6배 메밀 Bestco 희석용 소바 국수장국,2023,15060,16160
...,...,...,...,...,...,...,...,...,...,...,...
10049,L5,음료,M25,탄산,S0630,탄산음료,L5-M25-S0630-1053,한국 탄산음료 코카콜라 1.5L,2022,7180,7790
10050,L5,음료,M25,탄산,S0630,탄산음료,L5-M25-S0630-1053,한국 탄산음료 코카콜라 1.5L,2021,6820,7320
10051,L5,음료,M25,탄산,S0630,탄산음료,L5-M25-S0630-1054,환타 오렌지 190ml x 30캔 / 탄산음료 음료수,2023,15890,17050
10052,L5,음료,M25,탄산,S0630,탄산음료,L5-M25-S0630-1054,환타 오렌지 190ml x 30캔 / 탄산음료 음료수,2022,14780,16370


In [46]:
df_users=pd.read_csv("./on_users.csv", encoding="cp949")
df_users

,idUser,Gender,Age,FamilyCount,MemberYN
0,U10001,여성,26,2,Y
1,U10002,남성,61,2,Y
2,U10003,여성,34,2,Y
3,U10004,남성,26,1,N
4,U10005,여성,33,3,Y
...,...,...,...,...,...
2995,U12996,여성,66,2,N
2996,U12997,남성,24,2,Y
2997,U12998,남성,31,2,Y
2998,U12999,여성,32,1,Y


In [47]:
import pandas as pd

# df_orders 예시 (df_orders는 이미 로드된 상태여야 합니다)
# df_orders = pd.read_csv('your_data.csv')  # 실제 데이터프레임 로드 부분

# idUser, idOrder 기준으로 묶고, 그에 대한 ItemCode를 리스트로 만듦
df_grouped = df_orders.groupby(['idUser', 'idOrder'])['ItemCode'].agg(list).reset_index()

# 결과 확인
print(df_grouped.head())


   idUser            idOrder  \
0  U10001  U10001-O2021-1001   
1  U10001  U10001-O2021-1002   
2  U10001  U10001-O2021-1003   
3  U10001  U10001-O2021-1004   
4  U10001  U10001-O2021-1005   

                                            ItemCode  
0  [L4-M17-S0530-1019, L4-M18-S0110-1012, L1-M15-...  
1  [L4-M17-S0530-1024, L1-M21-S0540-1082, L1-M15-...  
2  [L4-M12-S0640-1057, L4-M12-S0350-1002, L1-M11-...  
3  [L4-M12-S0200-1021, L3-M16-S0390-1051, L1-M11-...  
4  [L1-M11-S0340-1017, L4-M22-S0700-1019, L1-M15-...  


In [48]:
df_grouped

,idUser,idOrder,ItemCode
0,U10001,U10001-O2021-1001,"[L4-M17-S0530-1019, L4-M18-S0110-1012, L1-M15-..."
1,U10001,U10001-O2021-1002,"[L4-M17-S0530-1024, L1-M21-S0540-1082, L1-M15-..."
2,U10001,U10001-O2021-1003,"[L4-M12-S0640-1057, L4-M12-S0350-1002, L1-M11-..."
3,U10001,U10001-O2021-1004,"[L4-M12-S0200-1021, L3-M16-S0390-1051, L1-M11-..."
4,U10001,U10001-O2021-1005,"[L1-M11-S0340-1017, L4-M22-S0700-1019, L1-M15-..."
...,...,...,...
171965,U13000,U13000-O2023-1016,"[L4-M22-S0700-1054, L4-M22-S0360-1062, L4-M12-..."
171966,U13000,U13000-O2023-1017,"[L4-M22-S0700-1061, L1-M12-S0200-1009, L1-M21-..."
171967,U13000,U13000-O2023-1018,"[L5-M20-S0720-1042, L4-M22-S0680-1025, L4-M22-..."
171968,U13000,U13000-O2023-1019,"[L4-M17-S0130-1003, L4-M12-S0350-1099, L1-M23-..."


In [50]:
import pandas as pd

# df_user에서 idUser, Gender, Age 정보를 딕셔너리로 변환
user_dict = df_users.set_index('idUser')[['Gender', 'Age']].to_dict(orient='index')

# df_orders에서 idUser, idOrder 기준으로 묶고, ItemCode를 리스트로 만듦
df_grouped = df_orders.groupby(['idUser', 'idOrder'])['ItemCode'].agg(list).reset_index()

# df_grouped에 Gender와 Age 열 추가
df_grouped['Gender'] = df_grouped['idUser'].apply(lambda x: user_dict.get(x, {}).get('Gender', 'Unknown'))
df_grouped['Age'] = df_grouped['idUser'].apply(lambda x: user_dict.get(x, {}).get('Age', 'Unknown'))

# 결과 확인
print(df_grouped.head())


   idUser            idOrder  \
0  U10001  U10001-O2021-1001   
1  U10001  U10001-O2021-1002   
2  U10001  U10001-O2021-1003   
3  U10001  U10001-O2021-1004   
4  U10001  U10001-O2021-1005   

                                            ItemCode Gender  Age  
0  [L4-M17-S0530-1019, L4-M18-S0110-1012, L1-M15-...     여성   26  
1  [L4-M17-S0530-1024, L1-M21-S0540-1082, L1-M15-...     여성   26  
2  [L4-M12-S0640-1057, L4-M12-S0350-1002, L1-M11-...     여성   26  
3  [L4-M12-S0200-1021, L3-M16-S0390-1051, L1-M11-...     여성   26  
4  [L1-M11-S0340-1017, L4-M22-S0700-1019, L1-M15-...     여성   26  


In [53]:
df_grouped['Gender'].value_counts()

여성    103855
남성     68115
Name: Gender, dtype: int64

In [54]:
import pandas as pd

# 대분류 및 중분류 추출 함수
def extract_category_subcategory(itemcode):
    parts = itemcode.split("-")
    if len(parts) > 1:
        return parts[0], parts[2]  # 대분류, 중분류 리턴
    return None, None  # 없으면 None 리턴

# 20대 남성 데이터 필터링 함수
def filter_20s_men(df):
    # 20대 남성 필터링
    return df[(df['Age'] >= 20) & (df['Age'] < 30) & (df['Gender'] == '남성')]

# 20대 남성 고객들의 상위 아이템 소분류 및 아이템 코드 추출
def get_top_items_for_20s_men(df_orders, df_users):
    # 1. df_users에서 성별과 나이를 df_orders에 map으로 추가
    df_orders['Gender'] = df_orders['idUser'].map(df_users.set_index('idUser')['Gender'])
    df_orders['Age'] = df_orders['idUser'].map(df_users.set_index('idUser')['Age'])
    
    # 2. 20대 남성 데이터를 필터링
    filtered_df = filter_20s_men(df_orders)
    
    # 3. 아이템 코드에서 대분류, 중분류 추출
    filtered_df[['Category', 'Subcategory']] = filtered_df['ItemCode'].apply(lambda x: pd.Series(extract_category_subcategory(x)))
    
    # 4. 소분류별로 구매 건수 집계 (주문 건수 기준)
    top_subcategories = filtered_df['Subcategory'].value_counts().head(10)
    
    # 5. 아이템 코드별로 주문 건수 및 매출액 집계
    # 매출액 = 가격 * 주문 건수 (Price 열을 사용하여 매출액 계산)
    filtered_df['Revenue'] = filtered_df['Price']
    
    top_items = filtered_df.groupby('ItemCode').agg({'Revenue': 'sum', 'ItemCode': 'count'}) \
                           .rename(columns={'ItemCode': 'OrderCount'}) \
                           .sort_values(by='Revenue', ascending=False).head(10)

    return top_subcategories, top_items

# df_orders 예시 (df_orders는 이미 로드된 상태여야 합니다)
# df_orders = pd.read_csv('your_data.csv')  # 실제 데이터프레임 로드 부분
# df_users = pd.read_csv('your_user_data.csv')  # 사용자 데이터 로드 부분

# 20대 남성의 상위 아이템 소분류 및 아이템 코드 분석
top_subcategories, top_items = get_top_items_for_20s_men(df_orders, df_users)

# 결과 출력
print("\n20대 남성 대분류별 중분류 상위 10개:")
print(top_subcategories)

print("\n20대 남성 상위 아이템 코드 및 매출액 (주문 건수 기준):")
print(top_items)



20대 남성 대분류별 중분류 상위 10개:
S0140    8133
S0350    6261
S0020    5302
S0640    4609
S0110    4347
S0540    3995
S0600    3489
S0100    3455
S0030    3403
S0700    3064
Name: Subcategory, dtype: int64

20대 남성 상위 아이템 코드 및 매출액 (주문 건수 기준):
                   Revenue  OrderCount
ItemCode                              
L4-M18-S0110-1036  2871650          80
L4-M18-S0110-1077  2570690          72
L4-M12-S0640-1016  2543560          69
L4-M22-S0360-1058  2472190          65
L4-M12-S0430-1016  2428400          69
L4-M17-S0530-1013  2402310          71
L4-M17-S0030-1005  2398520          64
L4-M18-S0110-1072  2378360          73
L4-M12-S0350-1046  2344410          68
L4-M12-S0350-1002  2302880          68


/tmp/ipykernel_19048/2381107324.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df[['Category', 'Subcategory']] = filtered_df['ItemCode'].apply(lambda x: pd.Series(extract_category_subcategory(x)))
/tmp/ipykernel_19048/2381107324.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df[['Category', 'Subcategory']] = filtered_df['ItemCode'].apply(lambda x: pd.Series(extract_category_subcategory(x)))
/tmp/ipykernel_19048/2381107324.py:32: SettingWithCopyWarning: 
A value is trying to be 

In [7]:

# df_users에서 idUser를 기준으로 dict를 생성
user_info_dict = df_users.set_index('idUser')[['Gender', 'Age']].to_dict(orient='index')

df_orders['Gender'] = df_orders['idUser'].map(lambda x: user_info_dict.get(x, {}).get('Gender'))
df_orders['Age'] = df_orders['idUser'].map(lambda x: user_info_dict.get(x, {}).get('Age'))

In [8]:
df_orders

,idUser,idOrder,OrderDT,ItemCode,Price,DeliveryDT,Gender,Age
0,U10001,U10001-O2021-1002,07JAN21:17:08:51,L4-M17-S0530-1024,33310,08JAN2021:06:24:00,여성,26
1,U10001,U10001-O2021-1002,07JAN21:17:08:51,L1-M21-S0540-1082,3780,08JAN2021:06:24:00,여성,26
2,U10001,U10001-O2021-1002,07JAN21:17:08:51,L1-M15-S0140-1311,22520,08JAN2021:06:24:00,여성,26
3,U10001,U10001-O2021-1002,07JAN21:17:08:51,L4-M12-S0350-1035,21630,08JAN2021:06:24:00,여성,26
4,U10001,U10001-O2021-1003,14JAN21:16:50:14,L4-M12-S0640-1057,11700,15JAN2021:06:28:00,여성,26
...,...,...,...,...,...,...,...,...
856618,U13000,U13000-O2023-1008,28DEC23:14:06:13,L4-M12-S0350-1063,11520,29DEC2023:04:23:00,여성,30
856619,U13000,U13000-O2023-1008,28DEC23:14:06:13,L1-M12-S0330-1011,18870,29DEC2023:04:23:00,여성,30
856620,U13000,U13000-O2023-1008,28DEC23:14:06:13,L1-M21-S0540-1161,3120,29DEC2023:04:23:00,여성,30
856621,U13000,U13000-O2023-1008,28DEC23:14:06:13,L1-M17-S0030-1016,37760,29DEC2023:04:23:00,여성,30


In [9]:
# 1. pd.cut()을 사용하여 Age를 연령대 범주로 변환
bins = [0, 19, 29, 39, 49, 59, 100]  # 나이 구간 설정
labels = ['10대', '20대', '30대', '40대', '50대', '60대 이상']  # 구간에 대한 라벨

df_orders['AgeGroup'] = pd.cut(df_orders['Age'], bins=bins, labels=labels, right=False)

# 2. Gender와 AgeGroup을 기준으로 그룹화
grouped = df_orders.groupby(['Gender', 'AgeGroup']).agg({
    'Price': 'mean',  # 평균 가격
    'idOrder': 'count'  # 주문 수
}).reset_index()

# 결과 출력
print(grouped)

   Gender AgeGroup         Price  idOrder
0      남성      10대           NaN        0
1      남성      20대  15404.446454    95123
2      남성      30대  15410.995962   146361
3      남성      40대  15468.341274    53734
4      남성      50대  15463.579967    31788
5      남성   60대 이상  15551.865294    11848
6      여성      10대           NaN        0
7      여성      20대  15405.090409   113706
8      여성      30대  15453.129268   211305
9      여성      40대  15404.502279   101382
10     여성      50대  15408.895174    60109
11     여성   60대 이상  15383.865737    31267


In [10]:
grouped

,Gender,AgeGroup,Price,idOrder
0,남성,10대,NaN,0
1,남성,20대,15404.446454,95123
2,남성,30대,15410.995962,146361
3,남성,40대,15468.341274,53734
4,남성,50대,15463.579967,31788
5,남성,60대 이상,15551.865294,11848
6,여성,10대,NaN,0
7,여성,20대,15405.090409,113706
8,여성,30대,15453.129268,211305
9,여성,40대,15404.502279,101382


In [11]:
# 1. Gender와 AgeGroup별로 아이템 코드 리스트로 그룹화
item_grouped = df_orders.groupby(['Gender', 'AgeGroup'])['ItemCode'].agg(list).reset_index()

# 2. Gender와 AgeGroup별로 가격 평균 및 주문 수 그룹화
price_grouped = df_orders.groupby(['Gender', 'AgeGroup']).agg({
    'Price': 'mean',  # 평균 가격
    'idOrder': 'count'  # 주문 수
}).reset_index()

# 3. 두 데이터프레임 합치기
grouped = pd.merge(price_grouped, item_grouped, on=['Gender', 'AgeGroup'])

# 결과 출력
print(grouped)

   Gender AgeGroup         Price  idOrder  \
0      남성      10대           NaN        0   
1      남성      20대  15404.446454    95123   
2      남성      30대  15410.995962   146361   
3      남성      40대  15468.341274    53734   
4      남성      50대  15463.579967    31788   
5      남성   60대 이상  15551.865294    11848   
6      여성      10대           NaN        0   
7      여성      20대  15405.090409   113706   
8      여성      30대  15453.129268   211305   
9      여성      40대  15404.502279   101382   
10     여성      50대  15408.895174    60109   
11     여성   60대 이상  15383.865737    31267   

                                             ItemCode  
0                                                 NaN  
1   [L4-M12-S0350-1037, L1-M12-S0330-1020, L1-M13-...  
2   [L4-M22-S0360-1029, L4-M12-S0620-1012, L1-M17-...  
3   [L4-M17-S0130-1017, L1-M17-S0260-1004, L1-M15-...  
4   [L4-M17-S0030-1027, L4-M12-S0640-1022, L1-M23-...  
5   [L4-M22-S0650-1007, L1-M17-S0100-1146, L1-M13-...  
6                     

In [14]:
grouped

,Gender,AgeGroup,Price,idOrder,ItemCode
0,남성,10대,NaN,0,NaN
1,남성,20대,15404.446454,95123,"[L4-M12-S0350-1037, L1-M12-S0330-1020, L1-M13-..."
2,남성,30대,15410.995962,146361,"[L4-M22-S0360-1029, L4-M12-S0620-1012, L1-M17-..."
3,남성,40대,15468.341274,53734,"[L4-M17-S0130-1017, L1-M17-S0260-1004, L1-M15-..."
4,남성,50대,15463.579967,31788,"[L4-M17-S0030-1027, L4-M12-S0640-1022, L1-M23-..."
5,남성,60대 이상,15551.865294,11848,"[L4-M22-S0650-1007, L1-M17-S0100-1146, L1-M13-..."
6,여성,10대,NaN,0,NaN
7,여성,20대,15405.090409,113706,"[L4-M17-S0530-1024, L1-M21-S0540-1082, L1-M15-..."
8,여성,30대,15453.129268,211305,"[L4-M22-S0020-1062, L3-M16-S0390-1043, L1-M24-..."
9,여성,40대,15404.502279,101382,"[L4-M12-S0620-1047, L1-M23-S0420-1041, L1-M22-..."


In [15]:
# 연령대별로 남성, 여성으로 그룹화
grouped_by_age = grouped.groupby(['AgeGroup', 'Gender']).agg({
    'Price': 'mean',        # 평균 가격
    'idOrder': 'sum',       # 총 주문 수 (주문 수 합계)
    'ItemCode': 'sum'       # 아이템 코드 목록 합산 (리스트 병합)
}).reset_index()

# 결과 출력
print(grouped_by_age)

   AgeGroup Gender         Price  idOrder  \
0       10대     남성           NaN        0   
1       10대     여성           NaN        0   
2       20대     남성  15404.446454    95123   
3       20대     여성  15405.090409   113706   
4       30대     남성  15410.995962   146361   
5       30대     여성  15453.129268   211305   
6       40대     남성  15468.341274    53734   
7       40대     여성  15404.502279   101382   
8       50대     남성  15463.579967    31788   
9       50대     여성  15408.895174    60109   
10   60대 이상     남성  15551.865294    11848   
11   60대 이상     여성  15383.865737    31267   

                                             ItemCode  
0                                                   0  
1                                                   0  
2   [L4-M12-S0350-1037, L1-M12-S0330-1020, L1-M13-...  
3   [L4-M17-S0530-1024, L1-M21-S0540-1082, L1-M15-...  
4   [L4-M22-S0360-1029, L4-M12-S0620-1012, L1-M17-...  
5   [L4-M22-S0020-1062, L3-M16-S0390-1043, L1-M24-...  
6   [L4-M17-S0130-1017

In [16]:
grouped_by_age

,AgeGroup,Gender,Price,idOrder,ItemCode
0,10대,남성,NaN,0,0
1,10대,여성,NaN,0,0
2,20대,남성,15404.446454,95123,"[L4-M12-S0350-1037, L1-M12-S0330-1020, L1-M13-..."
3,20대,여성,15405.090409,113706,"[L4-M17-S0530-1024, L1-M21-S0540-1082, L1-M15-..."
4,30대,남성,15410.995962,146361,"[L4-M22-S0360-1029, L4-M12-S0620-1012, L1-M17-..."
5,30대,여성,15453.129268,211305,"[L4-M22-S0020-1062, L3-M16-S0390-1043, L1-M24-..."
6,40대,남성,15468.341274,53734,"[L4-M17-S0130-1017, L1-M17-S0260-1004, L1-M15-..."
7,40대,여성,15404.502279,101382,"[L4-M12-S0620-1047, L1-M23-S0420-1041, L1-M22-..."
8,50대,남성,15463.579967,31788,"[L4-M17-S0030-1027, L4-M12-S0640-1022, L1-M23-..."
9,50대,여성,15408.895174,60109,"[L1-M17-S0030-1002, L4-M22-S0020-1035, L1-M11-..."


In [ ]:
이제 여기서 itemcode를 카테고리 별로 추출해야하는데 split("-")[0]

In [17]:
# 10대를 제외하고 20대, 30대, 40대, 50대, 60대 이상으로 데이터프레임 분리
df_20s = grouped_by_age[grouped_by_age['AgeGroup'] == '20대']
df_30s = grouped_by_age[grouped_by_age['AgeGroup'] == '30대']
df_40s = grouped_by_age[grouped_by_age['AgeGroup'] == '40대']
df_50s = grouped_by_age[grouped_by_age['AgeGroup'] == '50대']
df_60s = grouped_by_age[grouped_by_age['AgeGroup'] == '60대 이상']

In [18]:
df_20s

,AgeGroup,Gender,Price,idOrder,ItemCode
2,20대,남성,15404.446454,95123,"[L4-M12-S0350-1037, L1-M12-S0330-1020, L1-M13-..."
3,20대,여성,15405.090409,113706,"[L4-M17-S0530-1024, L1-M21-S0540-1082, L1-M15-..."


In [19]:
# 함수 정의: ItemCode에서 대분류 추출
def extract_category(itemcodes):
    return [item.split("-")[0] for item in itemcodes]

# 각 데이터프레임에 대해 대분류 추출
df_20s['Category'] = df_20s['ItemCode'].apply(extract_category)
df_30s['Category'] = df_30s['ItemCode'].apply(extract_category)
df_40s['Category'] = df_40s['ItemCode'].apply(extract_category)
df_50s['Category'] = df_50s['ItemCode'].apply(extract_category)
df_60s['Category'] = df_60s['ItemCode'].apply(extract_category)

/tmp/ipykernel_16625/1391467701.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_20s['Category'] = df_20s['ItemCode'].apply(extract_category)
/tmp/ipykernel_16625/1391467701.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_30s['Category'] = df_30s['ItemCode'].apply(extract_category)
/tmp/ipykernel_16625/1391467701.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docume

In [20]:
df_20s

,AgeGroup,Gender,Price,idOrder,ItemCode,Category
2,20대,남성,15404.446454,95123,"[L4-M12-S0350-1037, L1-M12-S0330-1020, L1-M13-...","[L4, L1, L1, L4, L4, L1, L4, L4, L1, L1, L4, L..."
3,20대,여성,15405.090409,113706,"[L4-M17-S0530-1024, L1-M21-S0540-1082, L1-M15-...","[L4, L1, L1, L4, L4, L4, L1, L1, L4, L3, L1, L..."


In [21]:
def extract_subcategory(itemcodes):
    return [item.split("-")[1] for item in itemcodes]

# 각 데이터프레임에 대해 중분류 추출
df_20s['Subcategory'] = df_20s['ItemCode'].apply(extract_subcategory)
df_30s['Subcategory'] = df_30s['ItemCode'].apply(extract_subcategory)
df_40s['Subcategory'] = df_40s['ItemCode'].apply(extract_subcategory)
df_50s['Subcategory'] = df_50s['ItemCode'].apply(extract_subcategory)
df_60s['Subcategory'] = df_60s['ItemCode'].apply(extract_subcategory)

/tmp/ipykernel_16625/2582488271.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_20s['Subcategory'] = df_20s['ItemCode'].apply(extract_subcategory)
/tmp/ipykernel_16625/2582488271.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_30s['Subcategory'] = df_30s['ItemCode'].apply(extract_subcategory)
/tmp/ipykernel_16625/2582488271.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats i

In [22]:
df_20s

,AgeGroup,Gender,Price,idOrder,ItemCode,Category,Subcategory
2,20대,남성,15404.446454,95123,"[L4-M12-S0350-1037, L1-M12-S0330-1020, L1-M13-...","[L4, L1, L1, L4, L4, L1, L4, L4, L1, L1, L4, L...","[M12, M12, M13, M17, M12, M21, M22, M18, M21, ..."
3,20대,여성,15405.090409,113706,"[L4-M17-S0530-1024, L1-M21-S0540-1082, L1-M15-...","[L4, L1, L1, L4, L4, L4, L1, L1, L4, L3, L1, L...","[M17, M21, M15, M12, M12, M12, M11, M17, M12, ..."


In [23]:
df_20s_exploded = df_20s.explode('Category')

# 결과 출력
print("Exploded 20대 데이터프레임:")
print(df_20s_exploded[['AgeGroup', 'Gender', 'Category']])

Exploded 20대 데이터프레임:
   AgeGroup Gender Category
2       20대     남성       L4
2       20대     남성       L1
2       20대     남성       L1
2       20대     남성       L4
2       20대     남성       L4
..      ...    ...      ...
3       20대     여성       L1
3       20대     여성       L1
3       20대     여성       L1
3       20대     여성       L1
3       20대     여성       L1

[208829 rows x 3 columns]


In [24]:
df_20s_exploded

,AgeGroup,Gender,Price,idOrder,ItemCode,Category,Subcategory
2,20대,남성,15404.446454,95123,"[L4-M12-S0350-1037, L1-M12-S0330-1020, L1-M13-...",L4,"[M12, M12, M13, M17, M12, M21, M22, M18, M21, ..."
2,20대,남성,15404.446454,95123,"[L4-M12-S0350-1037, L1-M12-S0330-1020, L1-M13-...",L1,"[M12, M12, M13, M17, M12, M21, M22, M18, M21, ..."
2,20대,남성,15404.446454,95123,"[L4-M12-S0350-1037, L1-M12-S0330-1020, L1-M13-...",L1,"[M12, M12, M13, M17, M12, M21, M22, M18, M21, ..."
2,20대,남성,15404.446454,95123,"[L4-M12-S0350-1037, L1-M12-S0330-1020, L1-M13-...",L4,"[M12, M12, M13, M17, M12, M21, M22, M18, M21, ..."
2,20대,남성,15404.446454,95123,"[L4-M12-S0350-1037, L1-M12-S0330-1020, L1-M13-...",L4,"[M12, M12, M13, M17, M12, M21, M22, M18, M21, ..."
...,...,...,...,...,...,...,...
3,20대,여성,15405.090409,113706,"[L4-M17-S0530-1024, L1-M21-S0540-1082, L1-M15-...",L1,"[M17, M21, M15, M12, M12, M12, M11, M17, M12, ..."
3,20대,여성,15405.090409,113706,"[L4-M17-S0530-1024, L1-M21-S0540-1082, L1-M15-...",L1,"[M17, M21, M15, M12, M12, M12, M11, M17, M12, ..."
3,20대,여성,15405.090409,113706,"[L4-M17-S0530-1024, L1-M21-S0540-1082, L1-M15-...",L1,"[M17, M21, M15, M12, M12, M12, M11, M17, M12, ..."
3,20대,여성,15405.090409,113706,"[L4-M17-S0530-1024, L1-M21-S0540-1082, L1-M15-...",L1,"[M17, M21, M15, M12, M12, M12, M11, M17, M12, ..."


In [25]:
df_30s_exploded = df_30s.explode('Category')
df_40s_exploded = df_40s.explode('Category')
df_50s_exploded = df_50s.explode('Category')
df_60s_exploded = df_60s.explode('Category')

In [26]:
df_50s_exploded

,AgeGroup,Gender,Price,idOrder,ItemCode,Category,Subcategory
8,50대,남성,15463.579967,31788,"[L4-M17-S0030-1027, L4-M12-S0640-1022, L1-M23-...",L4,"[M17, M12, M23, M13, M12, M22, M17, M12, M12, ..."
8,50대,남성,15463.579967,31788,"[L4-M17-S0030-1027, L4-M12-S0640-1022, L1-M23-...",L4,"[M17, M12, M23, M13, M12, M22, M17, M12, M12, ..."
8,50대,남성,15463.579967,31788,"[L4-M17-S0030-1027, L4-M12-S0640-1022, L1-M23-...",L1,"[M17, M12, M23, M13, M12, M22, M17, M12, M12, ..."
8,50대,남성,15463.579967,31788,"[L4-M17-S0030-1027, L4-M12-S0640-1022, L1-M23-...",L1,"[M17, M12, M23, M13, M12, M22, M17, M12, M12, ..."
8,50대,남성,15463.579967,31788,"[L4-M17-S0030-1027, L4-M12-S0640-1022, L1-M23-...",L1,"[M17, M12, M23, M13, M12, M22, M17, M12, M12, ..."
...,...,...,...,...,...,...,...
9,50대,여성,15408.895174,60109,"[L1-M17-S0030-1002, L4-M22-S0020-1035, L1-M11-...",L1,"[M17, M22, M11, M22, M19, M15, M17, M22, M22, ..."
9,50대,여성,15408.895174,60109,"[L1-M17-S0030-1002, L4-M22-S0020-1035, L1-M11-...",L1,"[M17, M22, M11, M22, M19, M15, M17, M22, M22, ..."
9,50대,여성,15408.895174,60109,"[L1-M17-S0030-1002, L4-M22-S0020-1035, L1-M11-...",L1,"[M17, M22, M11, M22, M19, M15, M17, M22, M22, ..."
9,50대,여성,15408.895174,60109,"[L1-M17-S0030-1002, L4-M22-S0020-1035, L1-M11-...",L1,"[M17, M22, M11, M22, M19, M15, M17, M22, M22, ..."


In [27]:

# 1. 20대 데이터프레임 나누기
df_20s_L1 = df_20s_exploded[df_20s_exploded['Category'] == 'L1']
df_20s_L4 = df_20s_exploded[df_20s_exploded['Category'] == 'L4']

# 2. 30대 데이터프레임 나누기
df_30s_L1 = df_30s_exploded[df_30s_exploded['Category'] == 'L1']
df_30s_L4 = df_30s_exploded[df_30s_exploded['Category'] == 'L4']

# 3. 40대 데이터프레임 나누기
df_40s_L1 = df_40s_exploded[df_40s_exploded['Category'] == 'L1']
df_40s_L4 = df_40s_exploded[df_40s_exploded['Category'] == 'L4']

# 4. 50대 데이터프레임 나누기
df_50s_L1 = df_50s_exploded[df_50s_exploded['Category'] == 'L1']
df_50s_L4 = df_50s_exploded[df_50s_exploded['Category'] == 'L4']

# 5. 60대 이상 데이터프레임 나누기
df_60s_L1 = df_60s_exploded[df_60s_exploded['Category'] == 'L1']
df_60s_L4 = df_60s_exploded[df_60s_exploded['Category'] == 'L4']

In [28]:
df_20s_L1

,AgeGroup,Gender,Price,idOrder,ItemCode,Category,Subcategory
2,20대,남성,15404.446454,95123,"[L4-M12-S0350-1037, L1-M12-S0330-1020, L1-M13-...",L1,"[M12, M12, M13, M17, M12, M21, M22, M18, M21, ..."
2,20대,남성,15404.446454,95123,"[L4-M12-S0350-1037, L1-M12-S0330-1020, L1-M13-...",L1,"[M12, M12, M13, M17, M12, M21, M22, M18, M21, ..."
2,20대,남성,15404.446454,95123,"[L4-M12-S0350-1037, L1-M12-S0330-1020, L1-M13-...",L1,"[M12, M12, M13, M17, M12, M21, M22, M18, M21, ..."
2,20대,남성,15404.446454,95123,"[L4-M12-S0350-1037, L1-M12-S0330-1020, L1-M13-...",L1,"[M12, M12, M13, M17, M12, M21, M22, M18, M21, ..."
2,20대,남성,15404.446454,95123,"[L4-M12-S0350-1037, L1-M12-S0330-1020, L1-M13-...",L1,"[M12, M12, M13, M17, M12, M21, M22, M18, M21, ..."
...,...,...,...,...,...,...,...
3,20대,여성,15405.090409,113706,"[L4-M17-S0530-1024, L1-M21-S0540-1082, L1-M15-...",L1,"[M17, M21, M15, M12, M12, M12, M11, M17, M12, ..."
3,20대,여성,15405.090409,113706,"[L4-M17-S0530-1024, L1-M21-S0540-1082, L1-M15-...",L1,"[M17, M21, M15, M12, M12, M12, M11, M17, M12, ..."
3,20대,여성,15405.090409,113706,"[L4-M17-S0530-1024, L1-M21-S0540-1082, L1-M15-...",L1,"[M17, M21, M15, M12, M12, M12, M11, M17, M12, ..."
3,20대,여성,15405.090409,113706,"[L4-M17-S0530-1024, L1-M21-S0540-1082, L1-M15-...",L1,"[M17, M21, M15, M12, M12, M12, M11, M17, M12, ..."


In [ ]:
# 각 데이터프레임 그룹화 후 explode하기
df_20s_L1_grouped = df_20s_L1.groupby(['AgeGroup', 'Gender', 'Price', 'Category'])['Subcategory'].apply(lambda x: x.explode()).reset_index()
df_20s_L4_grouped = df_20s_L4.groupby(['AgeGroup', 'Gender', 'Price', 'Category'])['Subcategory'].apply(lambda x: x.explode()).reset_index()

df_30s_L1_grouped = df_30s_L1.groupby(['AgeGroup', 'Gender', 'Price', 'Category'])['Subcategory'].apply(lambda x: x.explode()).reset_index()
df_30s_L4_grouped = df_30s_L4.groupby(['AgeGroup', 'Gender', 'Price', 'Category'])['Subcategory'].apply(lambda x: x.explode()).reset_index()

df_40s_L1_grouped = df_40s_L1.groupby(['AgeGroup', 'Gender', 'Price', 'Category'])['Subcategory'].apply(lambda x: x.explode()).reset_index()
df_40s_L4_grouped = df_40s_L4.groupby(['AgeGroup', 'Gender', 'Price', 'Category'])['Subcategory'].apply(lambda x: x.explode()).reset_index()

df_50s_L1_grouped = df_50s_L1.groupby(['AgeGroup', 'Gender', 'Price', 'Category'])['Subcategory'].apply(lambda x: x.explode()).reset_index()
df_50s_L4_grouped = df_50s_L4.groupby(['AgeGroup', 'Gender', 'Price', 'Category'])['Subcategory'].apply(lambda x: x.explode()).reset_index()

df_60s_L1_grouped = df_60s_L1.groupby(['AgeGroup', 'Gender', 'Price', 'Category'])['Subcategory'].apply(lambda x: x.explode()).reset_index()
df_60s_L4_grouped = df_60s_L4.groupby(['AgeGroup', 'Gender', 'Price', 'Category'])['Subcategory'].apply(lambda x: x.explode()).reset_index()

In [7]:
# OrderDT를 datetime 형식으로 변환 (포맷 지정)
df_orders['OrderDT'] = pd.to_datetime(df_orders['OrderDT'], format='%d%b%y:%H:%M:%S')

# 2023년 데이터만 추출
df_orders_2023 = df_orders[df_orders['OrderDT'].dt.year == 2023]

In [8]:
df_orders_2023 

,idUser,idOrder,OrderDT,ItemCode,Price,DeliveryDT
240,U10001,U10001-O2023-1015,2023-01-10 21:10:57,L4-M12-S0430-1035,34690,12JAN2023:04:23:00
241,U10001,U10001-O2023-1015,2023-01-10 21:10:57,L1-M24-S0600-1166,19130,12JAN2023:04:23:00
242,U10001,U10001-O2023-1015,2023-01-10 21:10:57,L1-M12-S0200-1010,4620,12JAN2023:04:23:00
243,U10001,U10001-O2023-1015,2023-01-10 21:10:57,L1-M13-S0440-1038,9170,12JAN2023:04:23:00
244,U10001,U10001-O2023-1016,2023-01-24 18:32:40,L1-M15-S0140-1112,25950,25JAN2023:06:27:00
...,...,...,...,...,...,...
856618,U13000,U13000-O2023-1008,2023-12-28 14:06:13,L4-M12-S0350-1063,11520,29DEC2023:04:23:00
856619,U13000,U13000-O2023-1008,2023-12-28 14:06:13,L1-M12-S0330-1011,18870,29DEC2023:04:23:00
856620,U13000,U13000-O2023-1008,2023-12-28 14:06:13,L1-M21-S0540-1161,3120,29DEC2023:04:23:00
856621,U13000,U13000-O2023-1008,2023-12-28 14:06:13,L1-M17-S0030-1016,37760,29DEC2023:04:23:00


In [9]:
# idUser와 idOrder로 그룹화하여 Price는 합계, ItemCode는 리스트로 집계
df_orders_2023_grouped = df_orders_2023.groupby(['idUser', 'idOrder']).agg(
    Price=('Price', 'sum'),
    ItemCode=('ItemCode', lambda x: list(x))  # ItemCode는 리스트로 집계
).reset_index()


In [10]:
df_orders_2023_grouped

,idUser,idOrder,Price,ItemCode
0,U10001,U10001-O2023-1001,93110,"[L4-M12-S0350-1031, L4-M17-S0800-1001, L1-M17-..."
1,U10001,U10001-O2023-1002,61310,"[L4-M22-S0650-1023, L1-M12-S0330-1020, L4-M22-..."
2,U10001,U10001-O2023-1003,73020,"[L4-M17-S0030-1021, L4-M22-S0680-1012, L4-M23-..."
3,U10001,U10001-O2023-1004,91000,"[L4-M12-S0430-1013, L1-M22-S0040-1013, L4-M22-..."
4,U10001,U10001-O2023-1005,84390,"[L4-M22-S0360-1059, L1-M12-S0330-1012, L1-M17-..."
...,...,...,...,...
62822,U13000,U13000-O2023-1016,100720,"[L4-M22-S0700-1054, L4-M22-S0360-1062, L4-M12-..."
62823,U13000,U13000-O2023-1017,46950,"[L4-M22-S0700-1061, L1-M12-S0200-1009, L1-M21-..."
62824,U13000,U13000-O2023-1018,18810,"[L5-M20-S0720-1042, L4-M22-S0680-1025, L4-M22-..."
62825,U13000,U13000-O2023-1019,74280,"[L4-M17-S0130-1003, L4-M12-S0350-1099, L1-M23-..."


In [11]:
# 1. df_users에서 idUser를 기준으로 Gender와 Age를 딕셔너리로 변환
user_info_dict = df_users.set_index('idUser')[['Gender', 'Age']].to_dict(orient='index')

# 2. df_orders_2023_grouped에 user_info_dict를 매핑하여 새로운 컬럼 추가
df_orders_2023['UserInfo'] = df_orders_2023['idUser'].map(user_info_dict)

# 결과 확인
print(df_orders_2023)

        idUser            idOrder             OrderDT           ItemCode  \
240     U10001  U10001-O2023-1015 2023-01-10 21:10:57  L4-M12-S0430-1035   
241     U10001  U10001-O2023-1015 2023-01-10 21:10:57  L1-M24-S0600-1166   
242     U10001  U10001-O2023-1015 2023-01-10 21:10:57  L1-M12-S0200-1010   
243     U10001  U10001-O2023-1015 2023-01-10 21:10:57  L1-M13-S0440-1038   
244     U10001  U10001-O2023-1016 2023-01-24 18:32:40  L1-M15-S0140-1112   
...        ...                ...                 ...                ...   
856618  U13000  U13000-O2023-1008 2023-12-28 14:06:13  L4-M12-S0350-1063   
856619  U13000  U13000-O2023-1008 2023-12-28 14:06:13  L1-M12-S0330-1011   
856620  U13000  U13000-O2023-1008 2023-12-28 14:06:13  L1-M21-S0540-1161   
856621  U13000  U13000-O2023-1008 2023-12-28 14:06:13  L1-M17-S0030-1016   
856622  U13000  U13000-O2023-1008 2023-12-28 14:06:13  L1-M15-S0140-1024   

        Price          DeliveryDT                     UserInfo  
240     34690  12JAN20

/tmp/ipykernel_19048/2815921654.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_orders_2023['UserInfo'] = df_orders_2023['idUser'].map(user_info_dict)


In [12]:
df_orders_2023_grouped

,idUser,idOrder,Price,ItemCode,UserInfo
0,U10001,U10001-O2023-1001,93110,"[L4-M12-S0350-1031, L4-M17-S0800-1001, L1-M17-...","{'Gender': '여성', 'Age': 26}"
1,U10001,U10001-O2023-1002,61310,"[L4-M22-S0650-1023, L1-M12-S0330-1020, L4-M22-...","{'Gender': '여성', 'Age': 26}"
2,U10001,U10001-O2023-1003,73020,"[L4-M17-S0030-1021, L4-M22-S0680-1012, L4-M23-...","{'Gender': '여성', 'Age': 26}"
3,U10001,U10001-O2023-1004,91000,"[L4-M12-S0430-1013, L1-M22-S0040-1013, L4-M22-...","{'Gender': '여성', 'Age': 26}"
4,U10001,U10001-O2023-1005,84390,"[L4-M22-S0360-1059, L1-M12-S0330-1012, L1-M17-...","{'Gender': '여성', 'Age': 26}"
...,...,...,...,...,...
62822,U13000,U13000-O2023-1016,100720,"[L4-M22-S0700-1054, L4-M22-S0360-1062, L4-M12-...","{'Gender': '여성', 'Age': 30}"
62823,U13000,U13000-O2023-1017,46950,"[L4-M22-S0700-1061, L1-M12-S0200-1009, L1-M21-...","{'Gender': '여성', 'Age': 30}"
62824,U13000,U13000-O2023-1018,18810,"[L5-M20-S0720-1042, L4-M22-S0680-1025, L4-M22-...","{'Gender': '여성', 'Age': 30}"
62825,U13000,U13000-O2023-1019,74280,"[L4-M17-S0130-1003, L4-M12-S0350-1099, L1-M23-...","{'Gender': '여성', 'Age': 30}"


In [13]:
df_30s_women = df_orders_2023_grouped[
    (df_orders_2023_grouped['UserInfo'].apply(lambda x: x['Gender'] == '여성')) &
    (df_orders_2023_grouped['UserInfo'].apply(lambda x: 30 <= x['Age'] < 40))
]

# 주문 건수와 가격 합 구하기
order_count = df_30s_women.shape[0]  # 주문 건수
price_sum = df_30s_women['Price'].sum()  # 가격 합

# 결과 출력
print(f"30대 여성의 주문 건수: {order_count}")
print(f"30대 여성의 가격 합: {price_sum}")

30대 여성의 주문 건수: 15676
30대 여성의 가격 합: 1454509850


In [14]:
df_30s_women

,idUser,idOrder,Price,ItemCode,UserInfo
37,U10003,U10003-O2023-1001,47890,"[L1-M12-S0330-1003, L1-M11-S0660-1001, L1-M21-...","{'Gender': '여성', 'Age': 34}"
38,U10003,U10003-O2023-1002,146450,"[L4-M12-S0640-1069, L4-M17-S0500-1010, L4-M22-...","{'Gender': '여성', 'Age': 34}"
39,U10003,U10003-O2023-1003,81570,"[L5-M20-S0720-1080, L4-M17-S0030-1027, L1-M12-...","{'Gender': '여성', 'Age': 34}"
40,U10003,U10003-O2023-1004,36530,"[L4-M22-S0020-1049, L4-M17-S0810-1005, L1-M17-...","{'Gender': '여성', 'Age': 34}"
41,U10003,U10003-O2023-1005,99730,"[L4-M22-S0650-1011, L1-M24-S0600-1165, L4-M17-...","{'Gender': '여성', 'Age': 34}"
...,...,...,...,...,...
62822,U13000,U13000-O2023-1016,100720,"[L4-M22-S0700-1054, L4-M22-S0360-1062, L4-M12-...","{'Gender': '여성', 'Age': 30}"
62823,U13000,U13000-O2023-1017,46950,"[L4-M22-S0700-1061, L1-M12-S0200-1009, L1-M21-...","{'Gender': '여성', 'Age': 30}"
62824,U13000,U13000-O2023-1018,18810,"[L5-M20-S0720-1042, L4-M22-S0680-1025, L4-M22-...","{'Gender': '여성', 'Age': 30}"
62825,U13000,U13000-O2023-1019,74280,"[L4-M17-S0130-1003, L4-M12-S0350-1099, L1-M23-...","{'Gender': '여성', 'Age': 30}"


In [15]:
# 1. 아이템 코드에서 대분류와 중분류를 추출하는 함수
def extract_category(itemcode):
    return itemcode.split("-")[0]  # 대분류 추출

def extract_subcategory(itemcode):
    return itemcode.split("-")[1]  # 중분류 추출

# 2. df_30s_women에서 아이템 코드 추출
df_30s_women['Category'] = df_30s_women['ItemCode'].apply(lambda x: [extract_category(item) for item in x])
df_30s_women['Subcategory'] = df_30s_women['ItemCode'].apply(lambda x: [extract_subcategory(item) for item in x])

# 3. 대분류와 중분류의 빈도 계산
category_counts = df_30s_women['Category'].explode().value_counts()
subcategory_counts = df_30s_women['Subcategory'].explode().value_counts()


/tmp/ipykernel_16800/1957788430.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_30s_women['Category'] = df_30s_women['ItemCode'].apply(lambda x: [extract_category(item) for item in x])
/tmp/ipykernel_16800/1957788430.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_30s_women['Subcategory'] = df_30s_women['ItemCode'].apply(lambda x: [extract_subcategory(item) for item in x])


In [16]:
category_counts

L1    45596
L4    43561
L5     1474
L3      482
Name: Category, dtype: int64

In [17]:
subcategory_counts

M12    19184
M22    17155
M17    16962
M11     7568
M15     7074
M23     5174
M19     3859
M18     3744
M21     3530
M24     3063
M13     1844
M20     1054
M16      482
M25      420
Name: Subcategory, dtype: int64

In [18]:
# 2. df_30s_women에서 대분류와 중분류 컬럼을 새로 추가
df_30s_women['Category'] = df_30s_women['ItemCode'].apply(lambda x: [extract_category(item) for item in x])
df_30s_women['Subcategory'] = df_30s_women['ItemCode'].apply(lambda x: [extract_subcategory(item) for item in x])

# 3. 대분류별로 필터링하여 중분류 빈도 계산
# 먼저 Category와 Subcategory를 explode하여 리스트를 풀어줍니다.
df_30s_women_exploded = df_30s_women.explode('Category').explode('Subcategory')

# 4. L1 대분류에 속하는 중분류 5가지
l1_data = df_30s_women_exploded[df_30s_women_exploded['Category'] == 'L1']
l1_top_subcategories = l1_data['Subcategory'].value_counts().head(5)

# 5. L4 대분류에 속하는 중분류 5가지
l4_data = df_30s_women_exploded[df_30s_women_exploded['Category'] == 'L4']
l4_top_subcategories = l4_data['Subcategory'].value_counts().head(5)


/tmp/ipykernel_16800/1226516220.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_30s_women['Category'] = df_30s_women['ItemCode'].apply(lambda x: [extract_category(item) for item in x])
/tmp/ipykernel_16800/1226516220.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_30s_women['Subcategory'] = df_30s_women['ItemCode'].apply(lambda x: [extract_subcategory(item) for item in x])


In [21]:
df_30s_women_exploded 

,idUser,idOrder,Price,ItemCode,UserInfo,Category,Subcategory
37,U10003,U10003-O2023-1001,47890,"[L1-M12-S0330-1003, L1-M11-S0660-1001, L1-M21-...","{'Gender': '여성', 'Age': 34}",L1,M12
37,U10003,U10003-O2023-1001,47890,"[L1-M12-S0330-1003, L1-M11-S0660-1001, L1-M21-...","{'Gender': '여성', 'Age': 34}",L1,M11
37,U10003,U10003-O2023-1001,47890,"[L1-M12-S0330-1003, L1-M11-S0660-1001, L1-M21-...","{'Gender': '여성', 'Age': 34}",L1,M21
37,U10003,U10003-O2023-1001,47890,"[L1-M12-S0330-1003, L1-M11-S0660-1001, L1-M21-...","{'Gender': '여성', 'Age': 34}",L1,M15
37,U10003,U10003-O2023-1001,47890,"[L1-M12-S0330-1003, L1-M11-S0660-1001, L1-M21-...","{'Gender': '여성', 'Age': 34}",L1,M12
...,...,...,...,...,...,...,...
62826,U13000,U13000-O2023-1020,84900,"[L4-M23-S0930-1018, L4-M22-S0670-1010, L1-M12-...","{'Gender': '여성', 'Age': 30}",L1,M22
62826,U13000,U13000-O2023-1020,84900,"[L4-M23-S0930-1018, L4-M22-S0670-1010, L1-M12-...","{'Gender': '여성', 'Age': 30}",L1,M12
62826,U13000,U13000-O2023-1020,84900,"[L4-M23-S0930-1018, L4-M22-S0670-1010, L1-M12-...","{'Gender': '여성', 'Age': 30}",L1,M21
62826,U13000,U13000-O2023-1020,84900,"[L4-M23-S0930-1018, L4-M22-S0670-1010, L1-M12-...","{'Gender': '여성', 'Age': 30}",L1,M17


In [19]:
l1_top_subcategories

M12    58443
M17    55815
M22    50016
M11    26777
M15    24481
Name: Subcategory, dtype: int64

In [20]:
l4_top_subcategories

M12    57700
M22    54253
M17    49248
M11    21176
M15    19593
Name: Subcategory, dtype: int64

In [22]:
# 1. 연령대와 성별에 맞는 데이터를 필터링하는 함수
def filter_by_age_gender(df, age_group, gender):
    if age_group == '20대':
        age_condition = (df['UserInfo'].apply(lambda x: 20 <= x['Age'] < 30))
    elif age_group == '30대':
        age_condition = (df['UserInfo'].apply(lambda x: 30 <= x['Age'] < 40))
    elif age_group == '40대':
        age_condition = (df['UserInfo'].apply(lambda x: 40 <= x['Age'] < 50))
    elif age_group == '50대':
        age_condition = (df['UserInfo'].apply(lambda x: 50 <= x['Age'] < 60))
    else:
        age_condition = df['UserInfo'].apply(lambda x: x['Age'] >= 60)  # 60대 이상
    
    gender_condition = df['UserInfo'].apply(lambda x: x['Gender'] == gender)
    
    return df[age_condition & gender_condition]

# 2. 아이템 코드에서 대분류 및 중분류 추출 함수
def extract_category(itemcode):
    return itemcode.split("-")[0]

def extract_subcategory(itemcode):
    return itemcode.split("-")[1]

# 3. 결과 분석 함수 (대분류별 중분류 상위 5개)
def get_top_subcategories_by_age_gender(df, age_group, gender):
    # 데이터를 연령대 및 성별로 필터링
    filtered_df = filter_by_age_gender(df, age_group, gender)
    
    # 대분류와 중분류 추출
    filtered_df['Category'] = filtered_df['ItemCode'].apply(lambda x: [extract_category(item) for item in x])
    filtered_df['Subcategory'] = filtered_df['ItemCode'].apply(lambda x: [extract_subcategory(item) for item in x])
    
    # explode()로 풀어내기
    exploded_df = filtered_df.explode('Category').explode('Subcategory')

    # L1 대분류에 속하는 중분류 상위 5개
    l1_data = exploded_df[exploded_df['Category'] == 'L1']
    l1_top_subcategories = l1_data['Subcategory'].value_counts().head(5)

    # L4 대분류에 속하는 중분류 상위 5개
    l4_data = exploded_df[exploded_df['Category'] == 'L4']
    l4_top_subcategories = l4_data['Subcategory'].value_counts().head(5)

    # 결과 출력
    print(f"{age_group} {gender} 대분류별 중분류 상위 5개:")
    print("L1 대분류에 속하는 중분류 상위 5개:")
    print(l1_top_subcategories)
    print("\nL4 대분류에 속하는 중분류 상위 5개:")
    print(l4_top_subcategories)
    print("\n" + "="*50)

# 4. 전체 연령대와 성별에 대해 반복 수행
age_groups = ['20대', '30대', '40대', '50대', '60대 이상']
genders = ['남성', '여성']

# 5. 반복문을 통해 연령대 및 성별에 대해 분석
for age_group in age_groups:
    for gender in genders:
        get_top_subcategories_by_age_gender(df_orders_2023, age_group, gender)


KeyError: 'UserInfo'

In [37]:
df_orders_2023

,idUser,idOrder,OrderDT,ItemCode,Price,DeliveryDT,UserInfo,Gender,Age,AgeGroup
240,U10001,U10001-O2023-1015,2023-01-10 21:10:57,L4-M12-S0430-1035,34690,12JAN2023:04:23:00,"{'Gender': '여성', 'Age': 26}",여성,26,20대
241,U10001,U10001-O2023-1015,2023-01-10 21:10:57,L1-M24-S0600-1166,19130,12JAN2023:04:23:00,"{'Gender': '여성', 'Age': 26}",여성,26,20대
242,U10001,U10001-O2023-1015,2023-01-10 21:10:57,L1-M12-S0200-1010,4620,12JAN2023:04:23:00,"{'Gender': '여성', 'Age': 26}",여성,26,20대
243,U10001,U10001-O2023-1015,2023-01-10 21:10:57,L1-M13-S0440-1038,9170,12JAN2023:04:23:00,"{'Gender': '여성', 'Age': 26}",여성,26,20대
244,U10001,U10001-O2023-1016,2023-01-24 18:32:40,L1-M15-S0140-1112,25950,25JAN2023:06:27:00,"{'Gender': '여성', 'Age': 26}",여성,26,20대
...,...,...,...,...,...,...,...,...,...,...
856618,U13000,U13000-O2023-1008,2023-12-28 14:06:13,L4-M12-S0350-1063,11520,29DEC2023:04:23:00,"{'Gender': '여성', 'Age': 30}",여성,30,30대
856619,U13000,U13000-O2023-1008,2023-12-28 14:06:13,L1-M12-S0330-1011,18870,29DEC2023:04:23:00,"{'Gender': '여성', 'Age': 30}",여성,30,30대
856620,U13000,U13000-O2023-1008,2023-12-28 14:06:13,L1-M21-S0540-1161,3120,29DEC2023:04:23:00,"{'Gender': '여성', 'Age': 30}",여성,30,30대
856621,U13000,U13000-O2023-1008,2023-12-28 14:06:13,L1-M17-S0030-1016,37760,29DEC2023:04:23:00,"{'Gender': '여성', 'Age': 30}",여성,30,30대


In [24]:
# 1. df_orders_2023에 UserInfo 추가하기
# df_users에서 'idUser'를 기준으로 'Gender'와 'Age' 컬럼을 가져와서 df_orders_2023에 추가
user_info_dict = df_users.set_index('idUser')[['Gender', 'Age']].to_dict(orient='index')
df_orders_2023['UserInfo'] = df_orders_2023['idUser'].map(user_info_dict)

# 2. 연령대와 성별에 맞는 데이터를 필터링하는 함수
def filter_by_age_gender(df, age_group, gender):
    if age_group == '20대':
        age_condition = (df['UserInfo'].apply(lambda x: 20 <= x['Age'] < 30))
    elif age_group == '30대':
        age_condition = (df['UserInfo'].apply(lambda x: 30 <= x['Age'] < 40))
    elif age_group == '40대':
        age_condition = (df['UserInfo'].apply(lambda x: 40 <= x['Age'] < 50))
    elif age_group == '50대':
        age_condition = (df['UserInfo'].apply(lambda x: 50 <= x['Age'] < 60))
    else:
        age_condition = df['UserInfo'].apply(lambda x: x['Age'] >= 60)  # 60대 이상
    
    gender_condition = df['UserInfo'].apply(lambda x: x['Gender'] == gender)
    
    return df[age_condition & gender_condition]

# 3. 아이템 코드에서 대분류 및 중분류 추출 함수
def extract_category(itemcode):
    return itemcode.split("-")[0]

def extract_subcategory(itemcode):
    return itemcode.split("-")[1]

# 4. 결과 분석 함수 (대분류별 중분류 상위 5개)
def get_top_subcategories_by_age_gender(df, age_group, gender):
    # 데이터를 연령대 및 성별로 필터링
    filtered_df = filter_by_age_gender(df, age_group, gender)
    
    # 대분류와 중분류 추출
    filtered_df['Category'] = filtered_df['ItemCode'].apply(lambda x: [extract_category(item) for item in x])
    filtered_df['Subcategory'] = filtered_df['ItemCode'].apply(lambda x: [extract_subcategory(item) for item in x])
    
    # explode()로 풀어내기
    exploded_df = filtered_df.explode('Category').explode('Subcategory')

    # L1 대분류에 속하는 중분류 상위 5개
    l1_data = exploded_df[exploded_df['Category'] == 'L1']
    l1_top_subcategories = l1_data['Subcategory'].value_counts().head(5)

    # L4 대분류에 속하는 중분류 상위 5개
    l4_data = exploded_df[exploded_df['Category'] == 'L4']
    l4_top_subcategories = l4_data['Subcategory'].value_counts().head(5)

    # 결과 출력
    print(f"{age_group} {gender} 대분류별 중분류 상위 5개:")
    print("L1 대분류에 속하는 중분류 상위 5개:")
    print(l1_top_subcategories)
    print("\nL4 대분류에 속하는 중분류 상위 5개:")
    print(l4_top_subcategories)
    print("\n" + "="*50)

# 5. 전체 연령대와 성별에 대해 반복 수행
age_groups = ['20대', '30대', '40대', '50대', '60대 이상']
genders = ['남성', '여성']

# 6. 반복문을 통해 연령대 및 성별에 대해 분석
for age_group in age_groups:
    for gender in genders:
        get_top_subcategories_by_age_gender(df_orders_2023, age_group, gender)


/tmp/ipykernel_16800/815347914.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_orders_2023['UserInfo'] = df_orders_2023['idUser'].map(user_info_dict)
/tmp/ipykernel_16800/815347914.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['Category'] = filtered_df['ItemCode'].apply(lambda x: [extract_category(item) for item in x])


IndexError: list index out of range

In [27]:
# 2. 연령대와 성별에 맞는 데이터를 필터링하는 함수
def filter_by_age_gender(df, age_group, gender):
    if age_group == '20대':
        age_condition = (df['UserInfo'].apply(lambda x: 20 <= x['Age'] < 30))
    elif age_group == '30대':
        age_condition = (df['UserInfo'].apply(lambda x: 30 <= x['Age'] < 40))
    elif age_group == '40대':
        age_condition = (df['UserInfo'].apply(lambda x: 40 <= x['Age'] < 50))
    elif age_group == '50대':
        age_condition = (df['UserInfo'].apply(lambda x: 50 <= x['Age'] < 60))
    else:
        age_condition = df['UserInfo'].apply(lambda x: x['Age'] >= 60)  # 60대 이상
    
    gender_condition = df['UserInfo'].apply(lambda x: x['Gender'] == gender)
    
    return df[age_condition & gender_condition]

# 1. 아이템 코드에서 대분류 및 중분류 추출 함수
def extract_category(itemcode):
    # "-"로 분리할 수 있는지 확인
    if "-" in itemcode:
        return itemcode.split("-")[0]
    return None  # "-"가 없다면 None을 반환

def extract_subcategory(itemcode):
    # "-"로 분리할 수 있는지 확인
    if "-" in itemcode:
        parts = itemcode.split("-")
        if len(parts) > 1:
            return parts[1]
    return None  # "-"가 없다면 None을 반환

# 2. 결과 분석 함수 (대분류별 중분류 상위 5개)
def get_top_subcategories_by_age_gender(df, age_group, gender):
    # 데이터를 연령대 및 성별로 필터링
    filtered_df = filter_by_age_gender(df, age_group, gender)
    
    # 대분류와 중분류 추출
    filtered_df['Category'] = filtered_df['ItemCode'].apply(lambda x: [extract_category(item) for item in x if extract_category(item) is not None])
    filtered_df['Subcategory'] = filtered_df['ItemCode'].apply(lambda x: [extract_subcategory(item) for item in x if extract_subcategory(item) is not None])
    
    # explode()로 풀어내기
    exploded_df = filtered_df.explode('Category').explode('Subcategory')

    # L1 대분류에 속하는 중분류 상위 5개
    l1_data = exploded_df[exploded_df['Category'] == 'L1']
    l1_top_subcategories = l1_data['Subcategory'].value_counts().head(5)

    # L4 대분류에 속하는 중분류 상위 5개
    l4_data = exploded_df[exploded_df['Category'] == 'L4']
    l4_top_subcategories = l4_data['Subcategory'].value_counts().head(5)

    # 결과 출력
    print(f"{age_group} {gender} 대분류별 중분류 상위 5개:")
    print("L1 대분류에 속하는 중분류 상위 5개:")
    print(l1_top_subcategories)
    print("\nL4 대분류에 속하는 중분류 상위 5개:")
    print(l4_top_subcategories)
    print("\n" + "="*50)

# 3. 전체 연령대와 성별에 대해 반복 수행
age_groups = ['20대', '30대', '40대', '50대', '60대 이상']
genders = ['남성', '여성']

# 4. 반복문을 통해 연령대 및 성별에 대해 분석
for age_group in age_groups:
    for gender in genders:
        get_top_subcategories_by_age_gender(df_orders_2023, age_group, gender)


/tmp/ipykernel_16800/405367909.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['Category'] = filtered_df['ItemCode'].apply(lambda x: [extract_category(item) for item in x if extract_category(item) is not None])
/tmp/ipykernel_16800/405367909.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['Subcategory'] = filtered_df['ItemCode'].apply(lambda x: [extract_subcategory(item) for item in x if extract_subcategory(item) is not None])


20대 남성 대분류별 중분류 상위 5개:
L1 대분류에 속하는 중분류 상위 5개:
Series([], Name: Subcategory, dtype: int64)

L4 대분류에 속하는 중분류 상위 5개:
Series([], Name: Subcategory, dtype: int64)



/tmp/ipykernel_16800/405367909.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['Category'] = filtered_df['ItemCode'].apply(lambda x: [extract_category(item) for item in x if extract_category(item) is not None])
/tmp/ipykernel_16800/405367909.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['Subcategory'] = filtered_df['ItemCode'].apply(lambda x: [extract_subcategory(item) for item in x if extract_subcategory(item) is not None])


20대 여성 대분류별 중분류 상위 5개:
L1 대분류에 속하는 중분류 상위 5개:
Series([], Name: Subcategory, dtype: int64)

L4 대분류에 속하는 중분류 상위 5개:
Series([], Name: Subcategory, dtype: int64)



/tmp/ipykernel_16800/405367909.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['Category'] = filtered_df['ItemCode'].apply(lambda x: [extract_category(item) for item in x if extract_category(item) is not None])
/tmp/ipykernel_16800/405367909.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['Subcategory'] = filtered_df['ItemCode'].apply(lambda x: [extract_subcategory(item) for item in x if extract_subcategory(item) is not None])


...

/tmp/ipykernel_16800/405367909.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['Category'] = filtered_df['ItemCode'].apply(lambda x: [extract_category(item) for item in x if extract_category(item) is not None])
/tmp/ipykernel_16800/405367909.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['Subcategory'] = filtered_df['ItemCode'].apply(lambda x: [extract_subcategory(item) for item in x if extract_subcategory(item) is not None])


50대 여성 대분류별 중분류 상위 5개:
L1 대분류에 속하는 중분류 상위 5개:
Series([], Name: Subcategory, dtype: int64)

L4 대분류에 속하는 중분류 상위 5개:
Series([], Name: Subcategory, dtype: int64)



/tmp/ipykernel_16800/405367909.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['Category'] = filtered_df['ItemCode'].apply(lambda x: [extract_category(item) for item in x if extract_category(item) is not None])
/tmp/ipykernel_16800/405367909.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['Subcategory'] = filtered_df['ItemCode'].apply(lambda x: [extract_subcategory(item) for item in x if extract_subcategory(item) is not None])


60대 이상 남성 대분류별 중분류 상위 5개:
L1 대분류에 속하는 중분류 상위 5개:
Series([], Name: Subcategory, dtype: int64)

L4 대분류에 속하는 중분류 상위 5개:
Series([], Name: Subcategory, dtype: int64)

60대 이상 여성 대분류별 중분류 상위 5개:
L1 대분류에 속하는 중분류 상위 5개:
Series([], Name: Subcategory, dtype: int64)

L4 대분류에 속하는 중분류 상위 5개:
Series([], Name: Subcategory, dtype: int64)



/tmp/ipykernel_16800/405367909.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['Category'] = filtered_df['ItemCode'].apply(lambda x: [extract_category(item) for item in x if extract_category(item) is not None])
/tmp/ipykernel_16800/405367909.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['Subcategory'] = filtered_df['ItemCode'].apply(lambda x: [extract_subcategory(item) for item in x if extract_subcategory(item) is not None])


NameError: name 'filtered_df' is not defined

In [28]:
# 1. 연령대와 성별에 맞는 데이터를 필터링하는 함수
def filter_by_age_gender(df, age_group, gender):
    if age_group == '20대':
        age_condition = (df['UserInfo'].apply(lambda x: 20 <= x['Age'] < 30))
    elif age_group == '30대':
        age_condition = (df['UserInfo'].apply(lambda x: 30 <= x['Age'] < 40))
    elif age_group == '40대':
        age_condition = (df['UserInfo'].apply(lambda x: 40 <= x['Age'] < 50))
    elif age_group == '50대':
        age_condition = (df['UserInfo'].apply(lambda x: 50 <= x['Age'] < 60))
    else:
        age_condition = df['UserInfo'].apply(lambda x: x['Age'] >= 60)  # 60대 이상
    
    gender_condition = df['UserInfo'].apply(lambda x: x['Gender'] == gender)
    
    return df[age_condition & gender_condition]

# 2. 아이템 코드에서 대분류 및 중분류 추출 함수
def extract_category(itemcode):
    # "-"로 분리할 수 있는지 확인
    if isinstance(itemcode, str) and "-" in itemcode:
        return itemcode.split("-")[0]
    return None  # "-"가 없다면 None을 반환

def extract_subcategory(itemcode):
    # "-"로 분리할 수 있는지 확인
    if isinstance(itemcode, str) and "-" in itemcode:
        parts = itemcode.split("-")
        if len(parts) > 1:
            return parts[1]
    return None  # "-"가 없다면 None을 반환

# 3. 결과 분석 함수 (대분류별 중분류 상위 5개)
def get_top_subcategories_by_age_gender(df, age_group, gender):
    # 데이터를 연령대 및 성별로 필터링
    filtered_df = filter_by_age_gender(df, age_group, gender)
    
    # 대분류와 중분류 추출
    # 각 ItemCode가 리스트 형식이 아닐 경우 리스트로 변경
    filtered_df['Category'] = filtered_df['ItemCode'].apply(lambda x: [extract_category(item) for item in x.split(',') if extract_category(item) is not None] if isinstance(x, str) else [])
    filtered_df['Subcategory'] = filtered_df['ItemCode'].apply(lambda x: [extract_subcategory(item) for item in x.split(',') if extract_subcategory(item) is not None] if isinstance(x, str) else [])
    
    # explode()로 풀어내기
    exploded_df = filtered_df.explode('Category').explode('Subcategory')

    # L1 대분류에 속하는 중분류 상위 5개
    l1_data = exploded_df[exploded_df['Category'] == 'L1']
    l1_top_subcategories = l1_data['Subcategory'].value_counts().head(5)

    # L4 대분류에 속하는 중분류 상위 5개
    l4_data = exploded_df[exploded_df['Category'] == 'L4']
    l4_top_subcategories = l4_data['Subcategory'].value_counts().head(5)

    # 결과 출력
    print(f"{age_group} {gender} 대분류별 중분류 상위 5개:")
    print("L1 대분류에 속하는 중분류 상위 5개:")
    print(l1_top_subcategories)
    print("\nL4 대분류에 속하는 중분류 상위 5개:")
    print(l4_top_subcategories)
    print("\n" + "="*50)

# 4. 전체 연령대와 성별에 대해 반복 수행
age_groups = ['20대', '30대', '40대', '50대', '60대 이상']
genders = ['남성', '여성']

# 5. 반복문을 통해 연령대 및 성별에 대해 분석
for age_group in age_groups:
    for gender in genders:
        get_top_subcategories_by_age_gender(df_orders_2023, age_group, gender)


/tmp/ipykernel_16800/3554747163.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['Category'] = filtered_df['ItemCode'].apply(lambda x: [extract_category(item) for item in x.split(',') if extract_category(item) is not None] if isinstance(x, str) else [])
/tmp/ipykernel_16800/3554747163.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['Subcategory'] = filtered_df['ItemCode'].apply(lambda x: [extract_subcategory(item) for item in x.split(',') if extract_subcategory(item) is not 

20대 남성 대분류별 중분류 상위 5개:
L1 대분류에 속하는 중분류 상위 5개:
M17    5542
M11    3655
M15    3477
M12    1998
M19    1972
Name: Subcategory, dtype: int64

L4 대분류에 속하는 중분류 상위 5개:
M22    8326
M12    7274
M17    2778
M18    1847
M23    1273
Name: Subcategory, dtype: int64



/tmp/ipykernel_16800/3554747163.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['Category'] = filtered_df['ItemCode'].apply(lambda x: [extract_category(item) for item in x.split(',') if extract_category(item) is not None] if isinstance(x, str) else [])
/tmp/ipykernel_16800/3554747163.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['Subcategory'] = filtered_df['ItemCode'].apply(lambda x: [extract_subcategory(item) for item in x.split(',') if extract_subcategory(item) is not 

20대 여성 대분류별 중분류 상위 5개:
L1 대분류에 속하는 중분류 상위 5개:
M17    7007
M11    4616
M15    4368
M12    2555
M19    2333
Name: Subcategory, dtype: int64

L4 대분류에 속하는 중분류 상위 5개:
M22    10064
M12     9077
M17     3342
M18     2169
M23     1541
Name: Subcategory, dtype: int64



/tmp/ipykernel_16800/3554747163.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['Category'] = filtered_df['ItemCode'].apply(lambda x: [extract_category(item) for item in x.split(',') if extract_category(item) is not None] if isinstance(x, str) else [])
/tmp/ipykernel_16800/3554747163.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['Subcategory'] = filtered_df['ItemCode'].apply(lambda x: [extract_subcategory(item) for item in x.split(',') if extract_subcategory(item) is not 

...

/tmp/ipykernel_16800/3554747163.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['Category'] = filtered_df['ItemCode'].apply(lambda x: [extract_category(item) for item in x.split(',') if extract_category(item) is not None] if isinstance(x, str) else [])
/tmp/ipykernel_16800/3554747163.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['Subcategory'] = filtered_df['ItemCode'].apply(lambda x: [extract_subcategory(item) for item in x.split(',') if extract_subcategory(item) is not 

50대 여성 대분류별 중분류 상위 5개:
L1 대분류에 속하는 중분류 상위 5개:
M17    2887
M11    1902
M15    1825
M12    1122
M19    1018
Name: Subcategory, dtype: int64

L4 대분류에 속하는 중분류 상위 5개:
M22    4188
M12    3838
M17    1502
M18     998
M23     670
Name: Subcategory, dtype: int64



/tmp/ipykernel_16800/3554747163.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['Category'] = filtered_df['ItemCode'].apply(lambda x: [extract_category(item) for item in x.split(',') if extract_category(item) is not None] if isinstance(x, str) else [])
/tmp/ipykernel_16800/3554747163.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['Subcategory'] = filtered_df['ItemCode'].apply(lambda x: [extract_subcategory(item) for item in x.split(',') if extract_subcategory(item) is not 

60대 이상 남성 대분류별 중분류 상위 5개:
L1 대분류에 속하는 중분류 상위 5개:
M17    507
M11    330
M15    314
M12    190
M19    180
Name: Subcategory, dtype: int64

L4 대분류에 속하는 중분류 상위 5개:
M22    735
M12    679
M17    244
M18    156
M23    112
Name: Subcategory, dtype: int64

60대 이상 여성 대분류별 중분류 상위 5개:
L1 대분류에 속하는 중분류 상위 5개:
M17    1403
M11     997
M15     899
M12     522
M19     490
Name: Subcategory, dtype: int64

L4 대분류에 속하는 중분류 상위 5개:
M22    2160
M12    1817
M17     703
M18     478
M23     339
Name: Subcategory, dtype: int64



/tmp/ipykernel_16800/3554747163.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['Category'] = filtered_df['ItemCode'].apply(lambda x: [extract_category(item) for item in x.split(',') if extract_category(item) is not None] if isinstance(x, str) else [])
/tmp/ipykernel_16800/3554747163.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['Subcategory'] = filtered_df['ItemCode'].apply(lambda x: [extract_subcategory(item) for item in x.split(',') if extract_subcategory(item) is not 

In [31]:
# 대분류 및 중분류 추출 함수
def extract_category_subcategory(itemcode):
    parts = itemcode.split("-")
    if len(parts) > 1:
        return parts[0], parts[1]  # 대분류, 중분류 리턴
    return None, None  # 없으면 None 리턴


In [32]:
# 연령대 및 성별로 데이터를 필터링하는 함수
def filter_by_age_gender(df, age_group, gender):
    age_ranges = {
        '20대': (20, 30),
        '30대': (30, 40),
        '40대': (40, 50),
        '50대': (50, 60),
        '60대 이상': (60, 200)
    }
    min_age, max_age = age_ranges[age_group]
    return df[(df['UserInfo'].apply(lambda x: min_age <= x['Age'] < max_age)) & 
              (df['UserInfo'].apply(lambda x: x['Gender'] == gender))]


In [33]:
def get_top_categories_and_subcategories(df, age_group, gender):
    # 1. 데이터를 연령대와 성별로 필터링
    filtered_df = filter_by_age_gender(df, age_group, gender)
    
    # 2. 대분류, 중분류 추출
    filtered_df[['Category', 'Subcategory']] = filtered_df['ItemCode'].apply(lambda x: pd.Series(extract_category_subcategory(x)))
    
    # 3. 대분류와 중분류 각각 상위 5개 추출
    top_categories = filtered_df['Category'].value_counts().head(5)
    top_subcategories = filtered_df['Subcategory'].value_counts().head(5)
    
    return top_categories, top_subcategories


In [34]:
age_groups = ['20대', '30대', '40대', '50대', '60대 이상']
genders = ['남성', '여성']

# 5. 연령대와 성별에 대해 반복하면서 상위 아이템 추출
top_categories_by_age_gender = {}  # 나중에 연관 분석에 사용할 상위 아이템들을 저장할 딕셔너리

for age_group in age_groups:
    for gender in genders:
        top_categories, top_subcategories = get_top_categories_and_subcategories(df_orders_2023, age_group, gender)
        top_categories_by_age_gender[(age_group, gender)] = {
            'TopCategories': top_categories,
            'TopSubcategories': top_subcategories
        }
        print(f"\n{age_group} {gender} 대분류 상위 5개:")
        print(top_categories)
        print(f"\n{age_group} {gender} 중분류 상위 5개:")
        print(top_subcategories)


/tmp/ipykernel_16800/2297076016.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df[['Category', 'Subcategory']] = filtered_df['ItemCode'].apply(lambda x: pd.Series(extract_category_subcategory(x)))
/tmp/ipykernel_16800/2297076016.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df[['Category', 'Subcategory']] = filtered_df['ItemCode'].apply(lambda x: pd.Series(extract_category_subcategory(x)))



20대 남성 대분류 상위 5개:
L1    22390
L4    21498
L5      697
L3      225
Name: Category, dtype: int64

20대 남성 중분류 상위 5개:
M12    9272
M22    8668
M17    8320
M11    3655
M15    3477
Name: Subcategory, dtype: int64


/tmp/ipykernel_16800/2297076016.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df[['Category', 'Subcategory']] = filtered_df['ItemCode'].apply(lambda x: pd.Series(extract_category_subcategory(x)))
/tmp/ipykernel_16800/2297076016.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df[['Category', 'Subcategory']] = filtered_df['ItemCode'].apply(lambda x: pd.Series(extract_category_subcategory(x)))



20대 여성 대분류 상위 5개:
L1    27732
L4    26193
L5      847
L3      334
Name: Category, dtype: int64

20대 여성 중분류 상위 5개:
M12    11632
M22    10449
M17    10349
M11     4616
M15     4368
Name: Subcategory, dtype: int64


/tmp/ipykernel_16800/2297076016.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df[['Category', 'Subcategory']] = filtered_df['ItemCode'].apply(lambda x: pd.Series(extract_category_subcategory(x)))
/tmp/ipykernel_16800/2297076016.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df[['Category', 'Subcategory']] = filtered_df['ItemCode'].apply(lambda x: pd.Series(extract_category_subcategory(x)))


...

/tmp/ipykernel_16800/2297076016.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df[['Category', 'Subcategory']] = filtered_df['ItemCode'].apply(lambda x: pd.Series(extract_category_subcategory(x)))
/tmp/ipykernel_16800/2297076016.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df[['Category', 'Subcategory']] = filtered_df['ItemCode'].apply(lambda x: pd.Series(extract_category_subcategory(x)))



50대 여성 대분류 상위 5개:
L1    11745
L4    11196
L5      389
L3      113
Name: Category, dtype: int64

50대 여성 중분류 상위 5개:
M12    4960
M17    4389
M22    4351
M11    1902
M15    1825
Name: Subcategory, dtype: int64


/tmp/ipykernel_16800/2297076016.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df[['Category', 'Subcategory']] = filtered_df['ItemCode'].apply(lambda x: pd.Series(extract_category_subcategory(x)))
/tmp/ipykernel_16800/2297076016.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df[['Category', 'Subcategory']] = filtered_df['ItemCode'].apply(lambda x: pd.Series(extract_category_subcategory(x)))



60대 이상 남성 대분류 상위 5개:
L1    2017
L4    1926
L5      58
L3      22
Name: Category, dtype: int64

60대 이상 남성 중분류 상위 5개:
M12    869
M22    763
M17    751
M11    330
M15    314
Name: Subcategory, dtype: int64

60대 이상 여성 대분류 상위 5개:
L1    5750
L4    5497
L5     199
L3      63
Name: Category, dtype: int64

60대 이상 여성 중분류 상위 5개:
M12    2339
M22    2248
M17    2106
M11     997
M15     899
Name: Subcategory, dtype: int64


/tmp/ipykernel_16800/2297076016.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df[['Category', 'Subcategory']] = filtered_df['ItemCode'].apply(lambda x: pd.Series(extract_category_subcategory(x)))
/tmp/ipykernel_16800/2297076016.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df[['Category', 'Subcategory']] = filtered_df['ItemCode'].apply(lambda x: pd.Series(extract_category_subcategory(x)))


In [35]:
from mlxtend.frequent_patterns import apriori, association_rules

# 연관 분석을 위한 이진 행렬 생성 함수
def create_binary_matrix(df, top_categories, top_subcategories):
    # 아이템 코드에 해당하는 대분류와 중분류만 선택
    filtered_df = df[df['Category'].isin(top_categories) & df['Subcategory'].isin(top_subcategories)]
    
    # 아이템 구매 이진 행렬 생성
    df_item_matrix = filtered_df['ItemCode'].apply(lambda x: [item for item in x.split(',') if item.split('-')[0] in top_categories and item.split('-')[1] in top_subcategories])
    df_item_matrix = df_item_matrix.apply(lambda x: {item: 1 for item in x})  # 각 아이템에 대해 1 표시
    df_item_matrix = pd.DataFrame(list(df_item_matrix)).fillna(0)  # NaN을 0으로 채움
    
    return df_item_matrix

# 연령대 및 성별별로 연관 분석을 수행
for age_group in age_groups:
    for gender in genders:
        # 상위 대분류, 중분류를 얻음
        top_categories = top_categories_by_age_gender[(age_group, gender)]['TopCategories'].index
        top_subcategories = top_categories_by_age_gender[(age_group, gender)]['TopSubcategories'].index
        
        # 이진 행렬 생성
        df_item_matrix = create_binary_matrix(df_orders_2023, top_categories, top_subcategories)
        
        # Apriori 알고리즘 적용
        frequent_itemsets = apriori(df_item_matrix, min_support=0.01, use_colnames=True)
        
        # 연관 규칙 추출
        rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)
        
        # 연관 규칙 출력
        print(f"\n{age_group} {gender} 연관 규칙:")
        print(rules)


KeyError: 'Category'

In [36]:
df_orders_2023

,idUser,idOrder,OrderDT,ItemCode,Price,DeliveryDT,UserInfo
240,U10001,U10001-O2023-1015,2023-01-10 21:10:57,L4-M12-S0430-1035,34690,12JAN2023:04:23:00,"{'Gender': '여성', 'Age': 26}"
241,U10001,U10001-O2023-1015,2023-01-10 21:10:57,L1-M24-S0600-1166,19130,12JAN2023:04:23:00,"{'Gender': '여성', 'Age': 26}"
242,U10001,U10001-O2023-1015,2023-01-10 21:10:57,L1-M12-S0200-1010,4620,12JAN2023:04:23:00,"{'Gender': '여성', 'Age': 26}"
243,U10001,U10001-O2023-1015,2023-01-10 21:10:57,L1-M13-S0440-1038,9170,12JAN2023:04:23:00,"{'Gender': '여성', 'Age': 26}"
244,U10001,U10001-O2023-1016,2023-01-24 18:32:40,L1-M15-S0140-1112,25950,25JAN2023:06:27:00,"{'Gender': '여성', 'Age': 26}"
...,...,...,...,...,...,...,...
856618,U13000,U13000-O2023-1008,2023-12-28 14:06:13,L4-M12-S0350-1063,11520,29DEC2023:04:23:00,"{'Gender': '여성', 'Age': 30}"
856619,U13000,U13000-O2023-1008,2023-12-28 14:06:13,L1-M12-S0330-1011,18870,29DEC2023:04:23:00,"{'Gender': '여성', 'Age': 30}"
856620,U13000,U13000-O2023-1008,2023-12-28 14:06:13,L1-M21-S0540-1161,3120,29DEC2023:04:23:00,"{'Gender': '여성', 'Age': 30}"
856621,U13000,U13000-O2023-1008,2023-12-28 14:06:13,L1-M17-S0030-1016,37760,29DEC2023:04:23:00,"{'Gender': '여성', 'Age': 30}"


In [38]:
import pandas as pd
import json  # json 모듈 추가

# 대분류 및 중분류 추출 함수
def extract_category_subcategory(itemcode):
    parts = itemcode.split("-")
    if len(parts) > 1:
        return parts[0], parts[1]  # 대분류, 중분류 리턴
    return None, None  # 없으면 None 리턴

# 20대 남성 데이터 필터링 함수
def filter_20s_men(df):
    # 20대 남성 필터링
    return df[
        (df['UserInfo'].apply(lambda x: isinstance(x, dict) and 20 <= x.get('Age', 0) < 30)) &  # Age가 20대인지 확인
        (df['UserInfo'].apply(lambda x: isinstance(x, dict) and x.get('Gender') == '남성'))  # Gender가 '남성'인지 확인
    ]

# 20대 남성 고객들의 상위 아이템 소분류 및 아이템 이름 추출
def get_top_items_for_20s_men(df):
    # UserInfo 컬럼이 문자열로 되어 있을 경우 이를 딕셔너리로 변환 (JSON 형식이라면)
    df['UserInfo'] = df['UserInfo'].apply(lambda x: json.loads(x) if isinstance(x, str) else x)
    
    # 1. 20대 남성 데이터를 필터링
    filtered_df = filter_20s_men(df)
    
    # 2. 아이템 코드에서 대분류, 중분류 추출
    filtered_df[['Category', 'Subcategory']] = filtered_df['ItemCode'].apply(lambda x: pd.Series(extract_category_subcategory(x)))
    
    # 3. 중분류별로 구매 건수 집계 (소분류)
    top_subcategories = filtered_df['Subcategory'].value_counts().head(5)
    
    # 4. 아이템 코드별로 주문 건수 집계
    top_items = filtered_df['ItemCode'].value_counts().head(5)
    
    return top_subcategories, top_items

# 예시 데이터 프레임
# df_orders = pd.read_csv('your_data.csv')  # 실제 데이터 로드 부분

# 20대 남성의 상위 아이템 소분류 및 아이템 코드 분석
top_subcategories, top_items = get_top_items_for_20s_men(df_orders_2023)

# 결과 출력
print("\n20대 남성 대분류별 중분류 상위 5개:")
print(top_subcategories)

print("\n20대 남성 상위 아이템 코드:")
print(top_items)


/tmp/ipykernel_19048/96852552.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['UserInfo'] = df['UserInfo'].apply(lambda x: json.loads(x) if isinstance(x, str) else x)



20대 남성 대분류별 중분류 상위 5개:
M12    9272
M22    8668
M17    8320
M11    3655
M15    3477
Name: Subcategory, dtype: int64

20대 남성 상위 아이템 코드:
L4-M12-S0200-1010    38
L4-M18-S0110-1065    38
L4-M22-S0700-1005    37
L4-M22-S0020-1055    36
L1-M17-S0230-1001    36
Name: ItemCode, dtype: int64


/tmp/ipykernel_19048/96852552.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df[['Category', 'Subcategory']] = filtered_df['ItemCode'].apply(lambda x: pd.Series(extract_category_subcategory(x)))
/tmp/ipykernel_19048/96852552.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df[['Category', 'Subcategory']] = filtered_df['ItemCode'].apply(lambda x: pd.Series(extract_category_subcategory(x)))


In [26]:
import pandas as pd

# 대분류 및 중분류 추출 함수
def extract_category_subcategory(itemcode):
    parts = itemcode.split("-")
    if len(parts) > 1:
        return parts[0], parts[1]  # 대분류, 중분류 리턴
    return None, None  # 없으면 None 리턴

# 40대 여성 데이터 필터링 함수
def filter_40s_women(df):
    # 40대 여성 필터링
    return df[(df['UserInfo'].apply(lambda x: 20 <= x['Age'] < 30)) & 
              (df['UserInfo'].apply(lambda x: x['Gender'] == '남성'))]

# 40대 여성 고객들의 상위 아이템 소분류 및 아이템 이름 추출
def get_top_items_for_40s_women(df):
    # 1. 40대 여성 데이터를 필터링
    filtered_df = filter_40s_women(df)
    
    # 2. 아이템 코드에서 대분류, 중분류 추출
    filtered_df[['Category', 'Subcategory']] = filtered_df['ItemCode'].apply(lambda x: pd.Series(extract_category_subcategory(x)))
    
    # 3. 중분류별로 구매 건수 집계 (소분류)
    top_subcategories = filtered_df['Subcategory'].value_counts().head(5)
    
    # 4. 아이템 코드별로 주문 건수 집계
    top_items = filtered_df['ItemCode'].value_counts().head(5)
    
    return top_subcategories, top_items

# df_orders 예시 (df_orders는 이미 로드된 상태여야 합니다)
# df_orders = pd.read_csv('your_data.csv')  # 실제 데이터프레임 로드 부분

# 40대 여성의 상위 아이템 소분류 및 아이템 코드 분석
top_subcategories, top_items = get_top_items_for_40s_women(df_orders_2023)

# 결과 출력
print("\n20대 남성 대분류별 중분류 상위 5개:")
print(top_subcategories)

print("\n20대 남성 상위 아이템 코드:")
print(top_items)



20대 남성 대분류별 중분류 상위 5개:
M12    9272
M22    8668
M17    8320
M11    3655
M15    3477
Name: Subcategory, dtype: int64

20대 남성 상위 아이템 코드:
L4-M12-S0200-1010    38
L4-M18-S0110-1065    38
L4-M22-S0700-1005    37
L4-M22-S0020-1055    36
L1-M17-S0230-1001    36
Name: ItemCode, dtype: int64


/tmp/ipykernel_19048/706681333.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df[['Category', 'Subcategory']] = filtered_df['ItemCode'].apply(lambda x: pd.Series(extract_category_subcategory(x)))
/tmp/ipykernel_19048/706681333.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df[['Category', 'Subcategory']] = filtered_df['ItemCode'].apply(lambda x: pd.Series(extract_category_subcategory(x)))


In [34]:
filtered_df

,idUser,idOrder,OrderDT,ItemCode,Price,DeliveryDT,UserInfo,Gender,Age,AgeGroup
17694,U10061,U10061-O2023-1008,2023-01-15 20:10:01,L4-M22-S0360-1061,33170,16JAN2023:06:37:00,"{'Gender': '여성', 'Age': 69}",여성,69,60대 이상
17695,U10061,U10061-O2023-1008,2023-01-15 20:10:01,L4-M17-S0030-1008,35650,16JAN2023:06:37:00,"{'Gender': '여성', 'Age': 69}",여성,69,60대 이상
17696,U10061,U10061-O2023-1008,2023-01-15 20:10:01,L4-M12-S0430-1017,26580,16JAN2023:06:37:00,"{'Gender': '여성', 'Age': 69}",여성,69,60대 이상
17697,U10061,U10061-O2023-1008,2023-01-15 20:10:01,L1-M21-S0540-1102,32930,16JAN2023:06:37:00,"{'Gender': '여성', 'Age': 69}",여성,69,60대 이상
17698,U10061,U10061-O2023-1008,2023-01-15 20:10:01,L1-M17-S0100-1033,16660,16JAN2023:06:37:00,"{'Gender': '여성', 'Age': 69}",여성,69,60대 이상
...,...,...,...,...,...,...,...,...,...,...
855445,U12996,U12996-O2023-1004,2023-12-29 18:56:12,L4-M22-S0360-1049,35930,30DEC2023:05:56:00,"{'Gender': '여성', 'Age': 66}",여성,66,60대 이상
855446,U12996,U12996-O2023-1004,2023-12-29 18:56:12,L3-M16-S0390-1032,18970,30DEC2023:05:56:00,"{'Gender': '여성', 'Age': 66}",여성,66,60대 이상
855447,U12996,U12996-O2023-1004,2023-12-29 18:56:12,L1-M21-S0540-1176,15610,30DEC2023:05:56:00,"{'Gender': '여성', 'Age': 66}",여성,66,60대 이상
855448,U12996,U12996-O2023-1004,2023-12-29 18:56:12,L1-M13-S0440-1021,3760,30DEC2023:05:56:00,"{'Gender': '여성', 'Age': 66}",여성,66,60대 이상


240    {'Gender': '여성', 'Age': 26}
241    {'Gender': '여성', 'Age': 26}
242    {'Gender': '여성', 'Age': 26}
243    {'Gender': '여성', 'Age': 26}
244    {'Gender': '여성', 'Age': 26}
                  ...             
335    {'Gender': '여성', 'Age': 26}
336    {'Gender': '여성', 'Age': 26}
337    {'Gender': '여성', 'Age': 26}
338    {'Gender': '여성', 'Age': 26}
339    {'Gender': '여성', 'Age': 26}
Name: UserInfo, Length: 100, dtype: object

In [31]:
import pandas as pd

# 대분류 및 중분류 추출 함수
def extract_category_subcategory(itemcode):
    parts = itemcode.split("-")
    if len(parts) > 1:
        return parts[0], parts[1]  # 대분류, 중분류 리턴
    return None, None  # 없으면 None 리턴

# 20대 남성 데이터 필터링 함수
def filter_20s_men(df):
    # 20대 남성 필터링
    return df[
        (df['UserInfo'].apply(lambda x: isinstance(x, dict) and 20 <= x.get('Age', 0) < 30)) &  # Age가 20대인지 확인
        (df['UserInfo'].apply(lambda x: isinstance(x, dict) and x.get('Gender') == '남성'))  # Gender가 '남성'인지 확인
    ]

# 20대 남성 고객들의 상위 아이템 소분류 및 아이템 이름 추출
def get_top_items_for_20s_men(df):
    # 1. 20대 남성 데이터를 필터링
    filtered_df = filter_20s_men(df)
    
    # 2. 아이템 코드에서 대분류, 중분류 추출
    filtered_df[['Category', 'Subcategory']] = filtered_df['ItemCode'].apply(lambda x: pd.Series(extract_category_subcategory(x)))
    
    # 3. 중분류별로 구매 건수 집계 (소분류)
    top_subcategories = filtered_df['Subcategory'].value_counts().head(5)
    
    # 4. 아이템 코드별로 주문 건수 집계
    top_items = filtered_df['ItemCode'].value_counts().head(5)
    
    return top_subcategories, top_items

# 예시 데이터 프레임
# df_orders = pd.read_csv('your_data.csv')  # 실제 데이터 로드 부분

# 20대 남성의 상위 아이템 소분류 및 아이템 코드 분석
top_subcategories, top_items = get_top_items_for_20s_men(df_orders_2023)

# 결과 출력
print("\n20대 남성 대분류별 중분류 상위 5개:")
print(top_subcategories)

print("\n20대 남성 상위 아이템 코드:")
print(top_items)



20대 남성 대분류별 중분류 상위 5개:
M12    9272
M22    8668
M17    8320
M11    3655
M15    3477
Name: Subcategory, dtype: int64

20대 남성 상위 아이템 코드:
L4-M12-S0200-1010    38
L4-M18-S0110-1065    38
L4-M22-S0700-1005    37
L4-M22-S0020-1055    36
L1-M17-S0230-1001    36
Name: ItemCode, dtype: int64


/tmp/ipykernel_19048/3633866497.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df[['Category', 'Subcategory']] = filtered_df['ItemCode'].apply(lambda x: pd.Series(extract_category_subcategory(x)))
/tmp/ipykernel_19048/3633866497.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df[['Category', 'Subcategory']] = filtered_df['ItemCode'].apply(lambda x: pd.Series(extract_category_subcategory(x)))


In [36]:
 filtered_df

,idUser,idOrder,OrderDT,ItemCode,Price,DeliveryDT,UserInfo,Gender,Age,AgeGroup
17694,U10061,U10061-O2023-1008,2023-01-15 20:10:01,L4-M22-S0360-1061,33170,16JAN2023:06:37:00,"{'Gender': '여성', 'Age': 69}",여성,69,60대 이상
17695,U10061,U10061-O2023-1008,2023-01-15 20:10:01,L4-M17-S0030-1008,35650,16JAN2023:06:37:00,"{'Gender': '여성', 'Age': 69}",여성,69,60대 이상
17696,U10061,U10061-O2023-1008,2023-01-15 20:10:01,L4-M12-S0430-1017,26580,16JAN2023:06:37:00,"{'Gender': '여성', 'Age': 69}",여성,69,60대 이상
17697,U10061,U10061-O2023-1008,2023-01-15 20:10:01,L1-M21-S0540-1102,32930,16JAN2023:06:37:00,"{'Gender': '여성', 'Age': 69}",여성,69,60대 이상
17698,U10061,U10061-O2023-1008,2023-01-15 20:10:01,L1-M17-S0100-1033,16660,16JAN2023:06:37:00,"{'Gender': '여성', 'Age': 69}",여성,69,60대 이상
...,...,...,...,...,...,...,...,...,...,...
855445,U12996,U12996-O2023-1004,2023-12-29 18:56:12,L4-M22-S0360-1049,35930,30DEC2023:05:56:00,"{'Gender': '여성', 'Age': 66}",여성,66,60대 이상
855446,U12996,U12996-O2023-1004,2023-12-29 18:56:12,L3-M16-S0390-1032,18970,30DEC2023:05:56:00,"{'Gender': '여성', 'Age': 66}",여성,66,60대 이상
855447,U12996,U12996-O2023-1004,2023-12-29 18:56:12,L1-M21-S0540-1176,15610,30DEC2023:05:56:00,"{'Gender': '여성', 'Age': 66}",여성,66,60대 이상
855448,U12996,U12996-O2023-1004,2023-12-29 18:56:12,L1-M13-S0440-1021,3760,30DEC2023:05:56:00,"{'Gender': '여성', 'Age': 66}",여성,66,60대 이상


In [23]:
# df_items에서 ItemCode와 ItemName을 딕셔너리로 변환
item_code_to_name = dict(zip(df_items['ItemCode'], df_items['ItemName']))

# top_items에서 아이템 코드에 해당하는 아이템 이름을 추가
top_items_with_names = top_items.reset_index()  # top_items를 DataFrame으로 변환
top_items_with_names['ItemName'] = top_items_with_names['index'].map(item_code_to_name)

# 결과 출력
print(top_items_with_names[['index', 'ItemName']])


               index                       ItemName
0  L4-M12-S0200-1010           농장딸기 논산설향딸기 생딸기 당일수확
1  L4-M18-S0110-1065  종가집 배추김치(소백남도) 1Kg 국산 남도 포기김치
2  L4-M22-S0700-1005               [맘스허브] 애호박1개  1개
3  L4-M22-S0020-1055        농협 햇 못난이 호박고구마 5kg 10kg
4  L1-M17-S0230-1001       D단대전통시장 중앙상회 국산 남해안 마른멸치


In [24]:
top_items_with_names

,index,ItemCode,ItemName
0,L4-M12-S0200-1010,38,농장딸기 논산설향딸기 생딸기 당일수확
1,L4-M18-S0110-1065,38,종가집 배추김치(소백남도) 1Kg 국산 남도 포기김치
2,L4-M22-S0700-1005,37,[맘스허브] 애호박1개 1개
3,L4-M22-S0020-1055,36,농협 햇 못난이 호박고구마 5kg 10kg
4,L1-M17-S0230-1001,36,D단대전통시장 중앙상회 국산 남해안 마른멸치


In [38]:
import pandas as pd

# 대분류 및 중분류 추출 함수
def extract_category_subcategory(itemcode):
    parts = itemcode.split("-")
    if len(parts) > 1:
        return parts[0], parts[1]  # 대분류, 중분류 리턴
    return None, None  # 없으면 None 리턴

# 40대 여성 데이터 필터링 함수
def filter_40s_women(df):
    # 40대 여성 필터링
    return df[(df['Age'] >= 40) & (df['Age'] < 50) & (df['Gender'] == '여성')]

# 40대 여성 고객들의 상위 아이템 소분류 및 아이템 코드 추출
def get_top_items_for_40s_women(df_orders, df_users):
    # 1. df_users에서 성별과 나이를 df_orders에 map으로 추가
    df_orders['Gender'] = df_orders['idUser'].map(df_users.set_index('idUser')['Gender'])
    df_orders['Age'] = df_orders['idUser'].map(df_users.set_index('idUser')['Age'])
    
    # 2. 40대 여성 데이터를 필터링
    filtered_df = filter_40s_women(df_orders)
    
    # 3. 아이템 코드에서 대분류, 중분류 추출
    filtered_df[['Category', 'Subcategory']] = filtered_df['ItemCode'].apply(lambda x: pd.Series(extract_category_subcategory(x)))
    
    # 4. 중분류별로 구매 건수 집계 (소분류)
    top_subcategories = filtered_df['Subcategory'].value_counts().head(5)
    
    # 5. 아이템 코드별로 주문 건수 집계
    top_items = filtered_df['ItemCode'].value_counts().head(5)
    
    return top_subcategories, top_items

# df_orders 예시 (df_orders는 이미 로드된 상태여야 합니다)
# df_orders = pd.read_csv('your_data.csv')  # 실제 데이터프레임 로드 부분
# df_users = pd.read_csv('your_user_data.csv')  # 사용자 데이터 로드 부분

# 40대 여성의 상위 아이템 소분류 및 아이템 코드 분석
top_subcategories, top_items = get_top_items_for_40s_women(df_orders, df_users)

# 결과 출력
print("\n40대 여성 대분류별 중분류 상위 5개:")
print(top_subcategories)

print("\n40대 여성 상위 아이템 코드:")
print(top_items)



40대 여성 대분류별 중분류 상위 5개:
M12    19877
M22    17893
M17    17284
M11     7800
M15     7215
Name: Subcategory, dtype: int64

40대 여성 상위 아이템 코드:
L1-M11-S0220-1030    84
L4-M18-S0110-1071    77
L4-M22-S0020-1020    74
L4-M17-S0500-1011    72
L4-M22-S0020-1014    72
Name: ItemCode, dtype: int64


/tmp/ipykernel_16800/853214815.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df[['Category', 'Subcategory']] = filtered_df['ItemCode'].apply(lambda x: pd.Series(extract_category_subcategory(x)))
/tmp/ipykernel_16800/853214815.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df[['Category', 'Subcategory']] = filtered_df['ItemCode'].apply(lambda x: pd.Series(extract_category_subcategory(x)))


In [39]:
top_items

L1-M11-S0220-1030    84
L4-M18-S0110-1071    77
L4-M22-S0020-1020    74
L4-M17-S0500-1011    72
L4-M22-S0020-1014    72
Name: ItemCode, dtype: int64

In [40]:
import pandas as pd

# 예시: df_items는 ItemCode와 ItemName을 포함하고 있어야 함
# df_items = pd.read_csv('items.csv')  # ItemCode와 ItemName이 포함된 데이터프레임 로드

# 40대 여성 데이터 필터링 함수
def filter_40s_women(df):
    # 40대 여성 필터링
    return df[(df['Age'] >= 40) & (df['Age'] < 50) & (df['Gender'] == '여성')]

# 대분류 및 중분류 추출 함수
def extract_category_subcategory(itemcode):
    parts = itemcode.split("-")
    if len(parts) > 1:
        return parts[0], parts[1]  # 대분류, 중분류 리턴
    return None, None  # 없으면 None 리턴

# 40대 여성 고객들의 상위 아이템 소분류 및 아이템 코드 추출
def get_top_items_for_40s_women(df_orders, df_users, df_items):
    # 1. df_users에서 성별과 나이를 df_orders에 map으로 추가
    df_orders['Gender'] = df_orders['idUser'].map(df_users.set_index('idUser')['Gender'])
    df_orders['Age'] = df_orders['idUser'].map(df_users.set_index('idUser')['Age'])
    
    # 2. 40대 여성 데이터를 필터링
    filtered_df = filter_40s_women(df_orders)
    
    # 3. 아이템 코드에서 대분류, 중분류 추출
    filtered_df[['Category', 'Subcategory']] = filtered_df['ItemCode'].apply(lambda x: pd.Series(extract_category_subcategory(x)))
    
    # 4. 중분류별로 구매 건수 집계 (소분류)
    top_subcategories = filtered_df['Subcategory'].value_counts().head(5)
    
    # 5. 아이템 코드별로 주문 건수 집계
    top_items = filtered_df['ItemCode'].value_counts().head(5)
    
    # 6. df_items에서 ItemCode와 ItemName을 dict로 변환
    item_code_to_name = dict(zip(df_items['ItemCode'], df_items['ItemName']))
    
    # 7. top_items에서 ItemCode를 ItemName으로 변환 (dict를 이용)
    top_items_with_names = top_items.index.to_frame(name='ItemCode')  # 인덱스를 DataFrame으로 변환
    top_items_with_names['ItemName'] = top_items_with_names['ItemCode'].map(item_code_to_name)
    
    return top_subcategories, top_items_with_names

# df_orders, df_users, df_items 예시 (df_orders, df_users, df_items는 이미 로드된 상태여야 합니다)
# df_orders = pd.read_csv('your_orders.csv')  # 실제 주문 데이터 로드
# df_users = pd.read_csv('your_user_data.csv')  # 사용자 데이터 로드
# df_items = pd.read_csv('your_items.csv')  # 아이템 데이터 로드 (ItemCode, ItemName)

# 40대 여성의 상위 아이템 소분류 및 아이템 코드 분석
top_subcategories, top_items_with_names = get_top_items_for_40s_women(df_orders, df_users, df_items)

# 결과 출력
print("\n40대 여성 대분류별 중분류 상위 5개:")
print(top_subcategories)

print("\n40대 여성 상위 아이템 코드 및 아이템 이름:")
print(top_items_with_names)



40대 여성 대분류별 중분류 상위 5개:
M12    19877
M22    17893
M17    17284
M11     7800
M15     7215
Name: Subcategory, dtype: int64

40대 여성 상위 아이템 코드 및 아이템 이름:
                            ItemCode  \
L1-M11-S0220-1030  L1-M11-S0220-1030   
L4-M18-S0110-1071  L4-M18-S0110-1071   
L4-M22-S0020-1020  L4-M22-S0020-1020   
L4-M17-S0500-1011  L4-M17-S0500-1011   
L4-M22-S0020-1014  L4-M22-S0020-1014   

                                                            ItemName  
L1-M11-S0220-1030  농심 신라면 120g 10개 + 얼큰한 너구리 120g 5개 + 올리브 짜파게티 140g  
L4-M18-S0110-1071                종가집 포기김치1kg x2개 + 총각김치900g x1개 (냉장)  
L4-M22-S0020-1020                   [뜰안애] 안면도 호박고구마 5kg(상100g-200g내)  
L4-M17-S0500-1011    해미래 [K쇼핑][해미래] 국내산 손질 냉동 오징어 270g 내외 (2마리) x 2팩  
L4-M22-S0020-1014                    [꿈꿀마을] 황토 해남꿀고구마 2kg 대 (300g이상)  


/tmp/ipykernel_16800/2203092836.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df[['Category', 'Subcategory']] = filtered_df['ItemCode'].apply(lambda x: pd.Series(extract_category_subcategory(x)))
/tmp/ipykernel_16800/2203092836.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df[['Category', 'Subcategory']] = filtered_df['ItemCode'].apply(lambda x: pd.Series(extract_category_subcategory(x)))


In [41]:
top_items_with_names

,ItemCode,ItemName
L1-M11-S0220-1030,L1-M11-S0220-1030,농심 신라면 120g 10개 + 얼큰한 너구리 120g 5개 + 올리브 짜파게티 140g
L4-M18-S0110-1071,L4-M18-S0110-1071,종가집 포기김치1kg x2개 + 총각김치900g x1개 (냉장)
L4-M22-S0020-1020,L4-M22-S0020-1020,[뜰안애] 안면도 호박고구마 5kg(상100g-200g내)
L4-M17-S0500-1011,L4-M17-S0500-1011,해미래 [K쇼핑][해미래] 국내산 손질 냉동 오징어 270g 내외 (2마리) x 2팩
L4-M22-S0020-1014,L4-M22-S0020-1014,[꿈꿀마을] 황토 해남꿀고구마 2kg 대 (300g이상)


In [19]:
import pandas as pd

# 예시 df_orders_2023 (UserInfo 컬럼에 딕셔너리 형태로 정보가 들어 있음)

# df_orders_2023: 고객 주문 데이터 (idUser, idOrder, ItemCode, UserInfo)
# UserInfo 컬럼에는 {'Gender': '여성', 'Age': 26}과 같은 딕셔너리가 있음

# 1. 연령대(AgeGroup) 계산 함수
def get_age_group(age):
    if 20 <= age < 30:
        return '20대'
    elif 30 <= age < 40:
        return '30대'
    elif 40 <= age < 50:
        return '40대'
    elif 50 <= age < 60:
        return '50대'
    else:
        return '60대 이상'

# 2. UserInfo에서 Gender와 Age를 추출하고 AgeGroup을 계산하여 새로운 컬럼에 추가
df_orders_2023['Gender'] = df_orders_2023['UserInfo'].apply(lambda x: x['Gender'])
df_orders_2023['Age'] = df_orders_2023['UserInfo'].apply(lambda x: x['Age'])
df_orders_2023['AgeGroup'] = df_orders_2023['Age'].apply(get_age_group)

# 3. 필터링 함수 (연령대 및 성별 기준으로)
def filter_by_age_gender(df, age_group, gender):
    return df[(df['AgeGroup'] == age_group) & (df['Gender'] == gender)]

# 4. 아이템 코드 리스트로 만들기 (고객별로)
def create_item_lists(df):
    item_lists = df.groupby('idUser')['ItemCode'].apply(list).reset_index()
    return item_lists

# 5. 아이템을 1,0으로 변환하여 연관 분석 준비

def encode_items(item_lists, all_items):
    encoded_df = pd.DataFrame()
    for item in all_items:
        # 각 사용자에 대해 아이템이 포함되어 있으면 1, 아니면 0을 매핑
        encoded_df[item] = item_lists['ItemCode'].apply(lambda x: 1 if item in x else 0)
    return encoded_df


# 6. 연관 분석 수행 함수 (Apriori 알고리즘 적용)
from mlxtend.frequent_patterns import apriori, association_rules

def apply_apriori(df, min_support=0.01, min_threshold=0.7):
    frequent_itemsets = apriori(df, min_support=min_support, use_colnames=True)
    rules = association_rules(frequent_itemsets, metric="lift", min_threshold=min_threshold)
    return rules

# 7. 연령대와 성별로 연관 분석 수행
age_groups = ['20대', '30대', '40대', '50대', '60대 이상']
genders = ['남성', '여성']

# 1. 아이템 수 제한 (상위 N개 아이템만 사용)
top_n_items = 25 # 상위 50개 아이템
all_items = df_orders_2023['ItemCode'].value_counts().head(top_n_items).index.tolist()

# 2. 연관 분석을 수행할 때 데이터 샘플링 (10% 샘플)
for age_group in age_groups:
    for gender in genders:
        print(f"연령대: {age_group}, 성별: {gender}")
        
        # 필터링된 데이터
        filtered_df = filter_by_age_gender(df_orders_2023, age_group, gender)
        
        # 10% 샘플링
        filtered_df_sample = filtered_df.sample(frac=0.1, random_state=42)
        
        # 고객별 아이템 리스트 생성
        item_lists = create_item_lists(filtered_df_sample)
        
        # 아이템을 인코딩 (1 또는 0으로)
        encoded_df = encode_items(item_lists, all_items)
        
        # 연관 분석 실행 (Apriori 알고리즘 적용)
        rules = apply_apriori(encoded_df, min_support=0.001, min_threshold=0.8)
        
        # 연관 규칙 출력
        print(rules[['antecedents', 'consequents', 'support', 'confidence', 'lift']].head())
        print("="*50)


/tmp/ipykernel_19048/3979078190.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_orders_2023['Gender'] = df_orders_2023['UserInfo'].apply(lambda x: x['Gender'])
/tmp/ipykernel_19048/3979078190.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_orders_2023['Age'] = df_orders_2023['UserInfo'].apply(lambda x: x['Age'])
/tmp/ipykernel_19048/3979078190.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value ins

연령대: 20대, 성별: 남성
           antecedents          consequents   support  confidence  lift
0  (L1-M17-S0230-1001)  (L4-M22-S0680-1010)  0.002717        0.25  46.0
1  (L4-M22-S0680-1010)  (L1-M17-S0230-1001)  0.002717        0.50  46.0
연령대: 20대, 성별: 여성


/home/piai/anaconda3/lib/python3.10/site-packages/mlxtend/frequent_patterns/fpcommon.py:109: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(
/home/piai/anaconda3/lib/python3.10/site-packages/mlxtend/frequent_patterns/fpcommon.py:109: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


           antecedents          consequents   support  confidence       lift
0  (L1-M11-S0220-1088)  (L4-M18-S0110-1074)  0.002212    0.500000  37.666667
1  (L4-M18-S0110-1074)  (L1-M11-S0220-1088)  0.002212    0.166667  37.666667
2  (L1-M17-S0230-1001)  (L4-M17-S0130-1033)  0.002212    0.142857  32.285714
3  (L4-M17-S0130-1033)  (L1-M17-S0230-1001)  0.002212    0.500000  32.285714
4  (L1-M17-S0230-1001)  (L4-M22-S0700-1018)  0.002212    0.142857  16.142857
연령대: 30대, 성별: 남성
           antecedents          consequents   support  confidence  lift
0  (L4-M17-S0530-1008)  (L1-M11-S0220-1030)  0.001873    0.333333  89.0
1  (L1-M11-S0220-1030)  (L4-M17-S0530-1008)  0.001873    0.500000  89.0
2  (L1-M11-S0220-1030)  (L4-M22-S0680-1010)  0.001873    0.500000  89.0
3  (L4-M22-S0680-1010)  (L1-M11-S0220-1030)  0.001873    0.333333  89.0
4  (L4-M22-S0020-1096)  (L4-M18-S0110-1074)  0.001873    0.200000  53.4
연령대: 30대, 성별: 여성
           antecedents          consequents  support  confidence      li

/home/piai/anaconda3/lib/python3.10/site-packages/mlxtend/frequent_patterns/fpcommon.py:109: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(
/home/piai/anaconda3/lib/python3.10/site-packages/mlxtend/frequent_patterns/fpcommon.py:109: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


           antecedents          consequents   support  confidence        lift
0  (L1-M11-S0220-1030)  (L4-M22-S0360-1030)  0.006452    1.000000  155.000000
1  (L4-M22-S0360-1030)  (L1-M11-S0220-1030)  0.006452    1.000000  155.000000
2  (L4-M18-S0110-1074)  (L4-M17-S0130-1033)  0.006452    0.333333   25.833333
3  (L4-M17-S0130-1033)  (L4-M18-S0110-1074)  0.006452    0.500000   25.833333
연령대: 40대, 성별: 여성
           antecedents          consequents   support  confidence       lift
0  (L4-M17-S0530-1016)  (L1-M11-S0220-1088)  0.003106    0.333333  21.466667
1  (L1-M11-S0220-1088)  (L4-M17-S0530-1016)  0.003106    0.200000  21.466667
연령대: 50대, 성별: 남성
           antecedents          consequents   support  confidence   lift
0  (L1-M11-S0220-1088)  (L4-M22-S0170-1023)  0.010101        1.00  24.75
1  (L4-M22-S0170-1023)  (L1-M11-S0220-1088)  0.010101        0.25  24.75
2  (L4-M12-S0290-1017)  (L4-M17-S0130-1007)  0.010101        0.50  24.75
3  (L4-M17-S0130-1007)  (L4-M12-S0290-1017)  0.010101

/home/piai/anaconda3/lib/python3.10/site-packages/mlxtend/frequent_patterns/fpcommon.py:109: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(
/home/piai/anaconda3/lib/python3.10/site-packages/mlxtend/frequent_patterns/fpcommon.py:109: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(
/home/piai/anaconda3/lib/python3.10/site-packages/mlxtend/frequent_patterns/fpcommon.py:109: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


           antecedents          consequents   support  confidence  lift
0  (L4-M18-S0110-1027)  (L4-M17-S0530-1016)  0.005208        1.00  48.0
1  (L4-M17-S0530-1016)  (L4-M18-S0110-1027)  0.005208        0.25  48.0
2  (L4-M18-S0110-1070)  (L4-M17-S0530-1008)  0.005208        1.00  96.0
3  (L4-M17-S0530-1008)  (L4-M18-S0110-1070)  0.005208        0.50  96.0
연령대: 60대 이상, 성별: 남성
           antecedents          consequents   support  confidence  lift
0  (L4-M17-S0530-1008)  (L4-M17-S0130-1033)  0.029412         1.0  34.0
1  (L4-M17-S0130-1033)  (L4-M17-S0530-1008)  0.029412         1.0  34.0
연령대: 60대 이상, 성별: 여성
Empty DataFrame
Columns: [antecedents, consequents, support, confidence, lift]
Index: []


/home/piai/anaconda3/lib/python3.10/site-packages/mlxtend/frequent_patterns/fpcommon.py:109: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(
/home/piai/anaconda3/lib/python3.10/site-packages/mlxtend/frequent_patterns/fpcommon.py:109: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


In [15]:
all_items

['L1-M11-S0220-1088',
 'L1-M11-S0220-1030',
 'L1-M17-S0230-1001',
 'L4-M12-S0290-1017',
 'L4-M18-S0110-1046',
 'L4-M18-S0110-1074',
 'L4-M17-S0500-1002',
 'L4-M17-S0530-1016',
 'L4-M17-S0530-1008',
 'L4-M17-S0810-1007',
 'L4-M22-S0170-1023',
 'L4-M17-S0130-1033',
 'L4-M17-S0130-1024',
 'L4-M17-S0130-1029',
 'L4-M17-S0130-1007',
 'L4-M22-S0700-1018',
 'L4-M18-S0110-1028',
 'L4-M22-S0020-1096',
 'L4-M18-S0110-1020',
 'L4-M18-S0110-1035',
 'L4-M22-S0680-1010',
 'L4-M18-S0110-1027',
 'L4-M17-S0030-1026',
 'L4-M22-S0360-1030',
 'L4-M18-S0110-1070',
 'L4-M18-S0110-1081',
 'L4-M18-S0110-1025',
 'L4-M18-S0110-1043',
 'L4-M17-S0530-1026',
 'L4-M17-S0500-1007',
 'L4-M12-S0350-1042',
 'L4-M22-S0650-1003',
 'L4-M12-S0640-1016',
 'L4-M22-S0020-1102',
 'L4-M17-S0500-1010',
 'L4-M22-S0670-1010',
 'L4-M22-S0700-1056',
 'L4-M17-S0030-1016',
 'L4-M12-S0640-1059',
 'L4-M22-S0680-1003',
 'L4-M17-S0530-1022',
 'L4-M18-S0110-1011',
 'L4-M17-S0030-1024',
 'L4-M22-S0360-1058',
 'L4-M18-S0110-1002',
 'L4-M12-S

In [16]:
item_lists

,idUser,ItemCode
0,U10061,"[L1-M21-S0540-1046, L4-M22-S0670-1020, L3-M16-..."
1,U10067,"[L1-M19-S0520-1063, L4-M18-S0110-1040, L1-M17-..."
2,U10083,"[L4-M12-S0620-1038, L4-M22-S0650-1010, L1-M15-..."
3,U10104,"[L1-M19-S0520-1073, L4-M22-S0670-1041, L1-M17-..."
4,U10150,"[L4-M17-S0130-1009, L1-M11-S0340-1041, L1-M11-..."
...,...,...
93,U12879,"[L1-M21-S0540-1070, L1-M22-S0040-1019, L4-M12-..."
94,U12904,"[L4-M12-S0430-1030, L4-M22-S0670-1015, L4-M12-..."
95,U12984,"[L1-M17-S0100-1123, L4-M22-S0670-1034, L1-M17-..."
96,U12993,"[L1-M12-S0330-1012, L4-M23-S0900-1011, L4-M22-..."


In [10]:
df_orders_2023

,idUser,idOrder,OrderDT,ItemCode,Price,DeliveryDT
240,U10001,U10001-O2023-1015,2023-01-10 21:10:57,L4-M12-S0430-1035,34690,12JAN2023:04:23:00
241,U10001,U10001-O2023-1015,2023-01-10 21:10:57,L1-M24-S0600-1166,19130,12JAN2023:04:23:00
242,U10001,U10001-O2023-1015,2023-01-10 21:10:57,L1-M12-S0200-1010,4620,12JAN2023:04:23:00
243,U10001,U10001-O2023-1015,2023-01-10 21:10:57,L1-M13-S0440-1038,9170,12JAN2023:04:23:00
244,U10001,U10001-O2023-1016,2023-01-24 18:32:40,L1-M15-S0140-1112,25950,25JAN2023:06:27:00
...,...,...,...,...,...,...
856618,U13000,U13000-O2023-1008,2023-12-28 14:06:13,L4-M12-S0350-1063,11520,29DEC2023:04:23:00
856619,U13000,U13000-O2023-1008,2023-12-28 14:06:13,L1-M12-S0330-1011,18870,29DEC2023:04:23:00
856620,U13000,U13000-O2023-1008,2023-12-28 14:06:13,L1-M21-S0540-1161,3120,29DEC2023:04:23:00
856621,U13000,U13000-O2023-1008,2023-12-28 14:06:13,L1-M17-S0030-1016,37760,29DEC2023:04:23:00
